<a href="https://colab.research.google.com/github/medaymenhassen/content/blob/main/dd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import mediapipe as mp_lib
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import os
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Configuration MediaPipe optimisée
MP_CONFIG = {
    'static_image_mode': False,
    'model_complexity': 2,
    'min_detection_confidence': 0.5,
    'min_tracking_confidence': 0.5,
    'enable_segmentation': True,
    'smooth_landmarks': True,
    'smooth_segmentation': False
}

# Points clés par exercice
exercises_points = {
    "t bar row":            [0, 11, 12, 13, 14, 15, 16, 23, 24],
    "pull up":              [0, 11, 12, 13, 14, 15, 16],
    "hammer curl":          [11, 12, 13, 14, 15, 16],
    "decline bench press":  [11, 12, 13, 14, 15, 16, 23, 24],
    "tricep pushdown":      [11, 12, 13, 14, 15, 16],
    "chest fly machine":    [11, 12, 13, 14, 15, 16],
    "squat":                [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],
    "bench press":          [11, 12, 13, 14, 15, 16, 23, 24],
    "push-up":              [0, 11, 12, 13, 14, 15, 16, 23, 24, 27, 28],
    "deadlift":             [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],
    "leg raises":           [0, 23, 24, 25, 26, 27, 28],
    "russian twist":        [0, 1, 11, 12, 15, 16, 23, 24],
    "barbell biceps curl":  [11, 12, 13, 14, 15, 16],
    "lateral raise":        [11, 12, 13, 14, 15, 16],
    "hip thrust":           [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],
    "lat pulldown":         [11, 12, 13, 14, 15, 16],
    "plank":                [0, 1, 11, 12, 23, 24, 27, 28],
    "incline bench press":  [11, 12, 13, 14, 15, 16, 23, 24],
    "tricep dips":          [11, 12, 13, 14, 15, 16],
    "leg extension":        [23, 24, 25, 26, 27, 28],
    "romanian deadlift":    [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],
    "shoulder press":       [0, 11, 12, 13, 14, 15, 16, 23, 24]
}

# Fonctions utilitaires robustes
def calculate_angle_3d(a, b, c):
    """Calcul d'angle 3D robuste avec gestion d'erreurs"""
    try:
        a, b, c = np.array(a), np.array(b), np.array(c)
        ba, bc = a - b, c - b

        norm_ba = np.linalg.norm(ba)
        norm_bc = np.linalg.norm(bc)

        if norm_ba < 1e-6 or norm_bc < 1e-6:
            return 0.0

        cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        return np.arccos(cosine_angle)
    except Exception as e:
        # Added detailed error reporting for debugging
        print(f"Error calculating angle: {e}, points: {a}, {b}, {c}")
        return 0.0

def calculate_distance(a, b):
    """Distance euclidienne robuste"""
    try:
        return np.linalg.norm(np.array(a) - np.array(b))
    except Exception as e:
        # Added detailed error reporting for debugging
        print(f"Error calculating distance: {e}, points: {a}, {b}")
        return 0.0

def calculate_velocity(prev, curr, dt=1.0):
    """Calcul de vitesse avec gestion d'erreurs"""
    try:
        return (np.array(curr) - np.array(prev)) / dt
    except Exception as e:
        # Added detailed error reporting for debugging
        print(f"Error calculating velocity: {e}, prev: {prev}, curr: {curr}")
        return np.zeros_like(prev)

def calculate_acceleration(prev_v, curr_v, dt=1.0):
    """Calcul d'accélération avec gestion d'erreurs"""
    try:
        return (np.array(curr_v) - np.array(prev_v)) / dt
    except Exception as e:
        # Added detailed error reporting for debugging
        print(f"Error calculating acceleration: {e}, prev_v: {prev_v}, curr_v: {curr_v}")
        return np.zeros_like(prev_v)

class OneEuroFilter:
    """Filtre One Euro pour lissage temporel avancé"""

    def __init__(self, freq=30.0, mincutoff=1.0, beta=0.007, dcutoff=1.0):
        self.freq = freq
        self.mincutoff = mincutoff
        self.beta = beta
        self.dcutoff = dcutoff
        self.x_prev = None
        self.dx_prev = None
        self.t_prev = None

    def __call__(self, x, timestamp=None):
        """Applique le filtrage One Euro"""
        if self.x_prev is None:
            self.x_prev = x
            self.t_prev = timestamp or 0
            return x

        # Calcul du delta temps
        if timestamp is not None and self.t_prev is not None:
            dt = timestamp - self.t_prev
            if dt <= 0:
                dt = 1.0 / self.freq
        else:
            dt = 1.0 / self.freq

        # Calcul de la dérivée
        dx = (x - self.x_prev) / dt

        if self.dx_prev is None:
            self.dx_prev = dx
        else:
            # Lissage de la dérivée
            alpha_d = self._alpha(self.dcutoff, dt)
            self.dx_prev = dx * alpha_d + self.dx_prev * (1 - alpha_d)

        # Calcul du cutoff adaptatif
        cutoff = self.mincutoff + self.beta * abs(self.dx_prev)
        alpha = self._alpha(cutoff, dt)

        # Lissage de la position
        x_filtered = x * alpha + self.x_prev * (1 - alpha)

        # Mise à jour des valeurs précédentes
        self.x_prev = x_filtered
        self.t_prev = timestamp

        return x_filtered

    def _alpha(self, cutoff, dt):
        """Calcule le facteur alpha pour le filtrage"""
        tau = 1.0 / (2 * np.pi * cutoff)
        return 1.0 / (1.0 + tau / dt)

    def reset(self):
        """Remet à zéro le filtre"""
        self.x_prev = None
        self.dx_prev = None
        self.t_prev = None

class PhysicsValidator:
    """Validateur de cohérence physique des poses"""

    def __init__(self):
        # Longueurs d'os moyennes normalisées
        self.bone_lengths = {
            'upper_arm_r': 0.28, 'forearm_r': 0.25,
            'upper_arm_l': 0.28, 'forearm_l': 0.25,
            'thigh_r': 0.42, 'shin_r': 0.40,
            'thigh_l': 0.42, 'shin_l': 0.40,
            'torso': 0.45, 'neck': 0.12
        }

        # Plages d'angles articulaires plausibles (radians)
        self.joint_angle_limits = {
            'elbow': (0.0, np.pi),
            'knee': (0.0, np.pi * 0.75),
            'shoulder': (0.0, np.pi),
            'hip': (0.0, np.pi)
        }

    def extract_landmarks_safely(self, landmarks):
        """Extraction sûre des landmarks vers structures Python natives"""
        safe_landmarks = []
        if landmarks:
            # Check if the input is a list of MediaPipe landmark objects
            if hasattr(landmarks[0], 'x'):
                for lm in landmarks:
                    safe_lm = {
                        'x': float(lm.x),
                        'y': float(lm.y),
                        'z': float(lm.z),
                        'visibility': float(getattr(lm, 'visibility', 1.0))
                    }
                    safe_landmarks.append(safe_lm)
            # Check if the input is already a list of dictionaries
            elif isinstance(landmarks[0], dict):
                 for lm in landmarks:
                    safe_lm = {
                        'x': float(lm.get('x', 0.0)),
                        'y': float(lm.get('y', 0.0)),
                        'z': float(lm.get('z', 0.0)),
                        'visibility': float(lm.get('visibility', 1.0))
                    }
                    safe_landmarks.append(safe_lm)
        return safe_landmarks


    def validate_bone_lengths(self, landmarks, selected_points):
        """Valide et corrige les longueurs d'os"""
        if not landmarks:
            return landmarks

        # Ensure landmarks are in the safe dictionary format
        safe_landmarks = self.extract_landmarks_safely(landmarks)

        def get_xyz(idx):
            if idx < len(safe_landmarks) and isinstance(safe_landmarks[idx], dict):
                lm = safe_landmarks[idx]
                return np.array([lm.get('x', 0.0), lm.get('y', 0.0), lm.get('z', 0.0)])
            return np.zeros(3)

        # Validation bras droit
        if all(idx in selected_points for idx in [12, 14, 16]):
            shoulder = get_xyz(12)
            elbow = get_xyz(14)
            wrist = get_xyz(16)

            # Correction longueur bras supérieur
            upper_arm_dir = elbow - shoulder
            upper_arm_norm = np.linalg.norm(upper_arm_dir)
            if upper_arm_norm > 1e-6:
                upper_arm_dir = upper_arm_dir / upper_arm_norm
                corrected_elbow = shoulder + upper_arm_dir * self.bone_lengths['upper_arm_r']

                if 14 < len(safe_landmarks) and isinstance(safe_landmarks[14], dict):
                     safe_landmarks[14]['x'] = float(corrected_elbow[0])
                     safe_landmarks[14]['y'] = float(corrected_elbow[1])
                     safe_landmarks[14]['z'] = float(corrected_elbow[2])

                # Correction avant-bras
                forearm_dir = wrist - corrected_elbow
                forearm_norm = np.linalg.norm(forearm_dir)
                if forearm_norm > 1e-6:
                    forearm_dir = forearm_dir / forearm_norm
                    corrected_wrist = corrected_elbow + forearm_dir * self.bone_lengths['forearm_r']

                    if 16 < len(safe_landmarks) and isinstance(safe_landmarks[16], dict):
                        safe_landmarks[16]['x'] = float(corrected_wrist[0])
                        safe_landmarks[16]['y'] = float(corrected_wrist[1])
                        safe_landmarks[16]['z'] = float(corrected_wrist[2])

        # Validation bras gauche
        if all(idx in selected_points for idx in [11, 13, 15]):
            shoulder = get_xyz(11)
            elbow = get_xyz(13)
            wrist = get_xyz(15)

            upper_arm_dir = elbow - shoulder
            upper_arm_norm = np.linalg.norm(upper_arm_dir)
            if upper_arm_norm > 1e-6:
                upper_arm_dir = upper_arm_dir / upper_arm_norm
                corrected_elbow = shoulder + upper_arm_dir * self.bone_lengths['upper_arm_l']

                if 13 < len(safe_landmarks) and isinstance(safe_landmarks[13], dict):
                    safe_landmarks[13]['x'] = float(corrected_elbow[0])
                    safe_landmarks[13]['y'] = float(corrected_elbow[1])
                    safe_landmarks[13]['z'] = float(corrected_elbow[2])

                forearm_dir = wrist - corrected_elbow
                forearm_norm = np.linalg.norm(forearm_dir)
                if forearm_norm > 1e-6:
                    forearm_dir = forearm_dir / forearm_norm
                    corrected_wrist = corrected_elbow + forearm_dir * self.bone_lengths['forearm_l']

                    if 15 < len(safe_landmarks) and isinstance(safe_landmarks[15], dict):
                        safe_landmarks[15]['x'] = float(corrected_wrist[0])
                        safe_landmarks[15]['y'] = float(corrected_wrist[1])
                        safe_landmarks[15]['z'] = float(corrected_wrist[2])

        # Validation jambe droite
        if all(idx in selected_points for idx in [24, 26, 28]):
            hip = get_xyz(24)
            knee = get_xyz(26)
            ankle = get_xyz(28)

            thigh_dir = knee - hip
            thigh_norm = np.linalg.norm(thigh_dir)
            if thigh_norm > 1e-6:
                thigh_dir = thigh_dir / thigh_norm
                corrected_knee = hip + thigh_dir * self.bone_lengths['thigh_r']

                if 26 < len(safe_landmarks) and isinstance(safe_landmarks[26], dict):
                    safe_landmarks[26]['x'] = float(corrected_knee[0])
                    safe_landmarks[26]['y'] = float(corrected_knee[1])
                    safe_landmarks[26]['z'] = float(corrected_knee[2])

                shin_dir = ankle - corrected_knee
                shin_norm = np.linalg.norm(shin_dir)
                if shin_norm > 1e-6:
                    shin_dir = shin_dir / shin_norm
                    corrected_ankle = corrected_knee + shin_dir * self.bone_lengths['shin_r']

                    if 28 < len(safe_landmarks) and isinstance(safe_landmarks[28], dict):
                        safe_landmarks[28]['x'] = float(corrected_ankle[0])
                        safe_landmarks[28]['y'] = float(corrected_ankle[1])
                        safe_landmarks[28]['z'] = float(corrected_ankle[2])

        return safe_landmarks

    def is_anatomically_plausible(self, landmarks, selected_points):
        """Vérifie si une pose est anatomiquement plausible"""
        try:
            # Ensure landmarks are in the safe dictionary format
            safe_landmarks = self.extract_landmarks_safely(landmarks)

            def get_xyz(idx):
                if idx < len(safe_landmarks) and isinstance(safe_landmarks[idx], dict):
                    lm = safe_landmarks[idx]
                    return [lm.get('x', 0.0), lm.get('y', 0.0), lm.get('z', 0.0)]
                return [0, 0, 0]

            # Vérification angles articulaires
            if all(idx in selected_points for idx in [12, 14, 16]):
                elbow_angle = calculate_angle_3d(get_xyz(12), get_xyz(14), get_xyz(16))
                if not (self.joint_angle_limits['elbow'][0] <= elbow_angle <= self.joint_angle_limits['elbow'][1]):
                    return False

            if all(idx in selected_points for idx in [24, 26, 28]):
                knee_angle = calculate_angle_3d(get_xyz(24), get_xyz(26), get_xyz(28))
                if not (self.joint_angle_limits['knee'][0] <= knee_angle <= self.joint_angle_limits['knee'][1]):
                    return False

            return True
        except Exception as e:
            # Added detailed error reporting for debugging
            print(f"Error during anatomical plausibility check: {e}")
            return False

class EnhancedExercisePreprocessor:
    """Preprocesseur amélioré avec contraintes physiques et lissage temporel"""

    def __init__(self, max_frames=45, step=2, use_physics_validation=True, use_temporal_smoothing=True):
        self.max_frames = max_frames
        self.step = step
        self.use_physics_validation = use_physics_validation
        self.use_temporal_smoothing = use_temporal_smoothing

        # Initialisation MediaPipe
        self.mp_pose = mp_lib.solutions.pose.Pose(**MP_CONFIG)

        # Initialisation des composants avancés
        if self.use_physics_validation:
            self.physics_validator = PhysicsValidator()

        if self.use_temporal_smoothing:
            self.temporal_filters = {}

    def _initialize_filters(self, exercise_name, selected_points):
        """Initialise les filtres One Euro pour chaque point clé"""
        if exercise_name not in self.temporal_filters:
            self.temporal_filters[exercise_name] = {}

            for point_idx in selected_points:
                for coord in ['x', 'y', 'z']:
                    filter_key = f"{point_idx}_{coord}"
                    self.temporal_filters[exercise_name][filter_key] = OneEuroFilter(
                        freq=30.0, mincutoff=1.0, beta=0.007, dcutoff=1.0
                    )

    def _apply_temporal_smoothing(self, landmarks, exercise_name, selected_points, timestamp=None):
        """Version corrigée du lissage temporel One Euro Filter"""
        if not self.use_temporal_smoothing or not landmarks:
            return landmarks

        self._initialize_filters(exercise_name, selected_points)

        # Always convert to safe dictionary format at the beginning
        safe_landmarks = self.physics_validator.extract_landmarks_safely(landmarks)

        # Application du filtrage sur les dictionnaires avec vérification
        for point_idx in selected_points:
            if point_idx < len(safe_landmarks) and isinstance(safe_landmarks[point_idx], dict):
                for coord in ['x', 'y', 'z']:
                    filter_key = f"{point_idx}_{coord}"
                    current_value = safe_landmarks[point_idx].get(coord, 0.0) # Use .get for safety
                    filtered_value = self.temporal_filters[exercise_name][filter_key](current_value, timestamp)

                    # Modification directe du dictionnaire
                    safe_landmarks[point_idx][coord] = float(filtered_value)

        return safe_landmarks

    def _generate_depth_hypotheses(self, landmarks, num_hypotheses=5):
        """Génère plusieurs hypothèses de profondeur pour gérer l'ambiguïté"""
        if not landmarks:
            return []

        hypotheses = []
        depth_scales = np.linspace(0.8, 1.2, num_hypotheses)

        for scale in depth_scales:
            hypothesis = []
            for lm in landmarks:
                adjusted_lm = {
                    'x': lm.get('x', 0.0),
                    'y': lm.get('y', 0.0),
                    'z': lm.get('z', 0.0) * scale,
                    'visibility': lm.get('visibility', 1.0)
                }
                hypothesis.append(adjusted_lm)
            hypotheses.append(hypothesis)

        return hypotheses

    def _select_best_hypothesis(self, hypotheses, exercise_name, selected_points):
        """Sélectionne l'hypothèse la plus plausible physiquement"""
        if not hypotheses:
            return None

        scores = []
        for hypothesis in hypotheses:
            score = 0.0

            # Score de plausibilité anatomique
            if self.physics_validator.is_anatomically_plausible(hypothesis, selected_points):
                score += 1.0

            # Score de cohérence des longueurs d'os
            try:
                def get_xyz(idx):
                    if idx < len(hypothesis) and isinstance(hypothesis[idx], dict):
                        lm = hypothesis[idx]
                        return [lm.get('x', 0.0), lm.get('y', 0.0), lm.get('z', 0.0)] # Use .get for safety
                    return [0, 0, 0]

                # Évaluation de la cohérence des bras
                if all(idx in selected_points for idx in [12, 14, 16]):
                    upper_arm_length = calculate_distance(get_xyz(12), get_xyz(14))
                    forearm_length = calculate_distance(get_xyz(14), get_xyz(16))

                    # Pénalité pour longueurs aberrantes
                    if 0.15 <= upper_arm_length <= 0.4 and 0.15 <= forearm_length <= 0.4:
                        score += 0.5

                scores.append(score)
            except Exception as e:
                 print(f"Error scoring hypothesis based on bone lengths: {e}")
                 scores.append(0.0)


        if not scores:
            return hypotheses[0]

        best_idx = np.argmax(scores)
        return hypotheses[best_idx]

    def process_video_enhanced(self, video_path, exercise_name):
        """Version corrigée du traitement vidéo avec gestion cohérente des types"""
        selected_points = exercises_points[exercise_name]
        expected_features = len(selected_points) * 3

        # Calculate the expected length of features per frame
        # Coordonnées 3D (selected_points * 3)
        # Angles (Assuming 3: elbow, knee, torso - need to match actual calculation)
        # Distances (Assuming 2: wrist-elbow, hip-knee - need to match actual calculation)
        # Vitesses (selected_points * 3)
        # Accélérations (selected_points * 3)
        # This calculation must match the feature generation logic precisely.
        # Based on the original code, it seems to be: selected_points*3 (coords) + 3 (angles) + 2 (distances) + selected_points*3 (vel) + selected_points*3 (accel)
        expected_length = (
            len(selected_points) * 3 +  # Coordonnées 3D
            3 +  # Angles (coude/genou/torse) - Corrected to 3 based on the feature extraction below
            2 +  # Distances (wrist-elbow, hip-knee) - Corrected to 2 based on the feature extraction below
            len(selected_points) * 3 +  # Vitesses
            len(selected_points) * 3    # Accélérations
        )


        # Variables pour le calcul de vitesse/accélération
        prev_positions = None
        prev_velocities = None

        # Ouverture de la vidéo
        cap = cv2.VideoCapture(video_path)
        frames = []
        current_frame = 0
        timestamp = 0
        processed_frames_count = 0
        landmarks_detected_count = 0

        print(f"--- Processing video: {Path(video_path).name} for exercise {exercise_name} ---")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if current_frame % self.step == 0:
                processed_frames_count += 1
                timestamp = current_frame / 30.0  # Assumer 30 FPS

                try:
                    # Conversion couleur et traitement MediaPipe
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = self.mp_pose.process(rgb_frame)

                    frame_data = []
                    current_positions = []
                    safe_landmarks = []

                    if results.pose_landmarks:
                        landmarks_detected_count += 1
                        landmarks = results.pose_landmarks.landmark

                        # Step 1: Convert MediaPipe landmarks to safe dictionary format
                        safe_landmarks = self.physics_validator.extract_landmarks_safely(landmarks)

                        # Étape 2: Lissage temporel (maintenant avec dictionnaires)
                        if self.use_temporal_smoothing and safe_landmarks:
                            safe_landmarks = self._apply_temporal_smoothing(
                                safe_landmarks, exercise_name, selected_points, timestamp
                            )

                        # Étape 3: Validation physique (avec dictionnaires)
                        if self.use_physics_validation and safe_landmarks:
                            hypotheses = self._generate_depth_hypotheses(safe_landmarks)
                            selected_hypothesis = self._select_best_hypothesis(hypotheses, exercise_name, selected_points)
                            if selected_hypothesis:
                                safe_landmarks = self.physics_validator.validate_bone_lengths(selected_hypothesis, selected_points)
                            else:
                                # If no plausible hypothesis found, treat as no landmarks detected
                                safe_landmarks = []


                        # Fonction d'extraction cohérente (always work with dictionaries)
                        def get_xyz(idx):
                            if idx < len(safe_landmarks) and isinstance(safe_landmarks[idx], dict):
                                lm = safe_landmarks[idx]
                                return [lm.get('x', 0.0), lm.get('y', 0.0), lm.get('z', 0.0)] # Use .get for safety
                            return [0.0, 0.0, 0.0]

                        # Extraction des coordonnées 3D pour les points SÉLECTIONNÉS
                        if safe_landmarks:
                            for idx in selected_points:
                                p = get_xyz(idx)
                                frame_data.extend(p)
                                current_positions.append(p)

                            # Calcul des angles (using selected points where applicable)
                            try:
                                # Angle coude droit (12, 14, 16)
                                if all(i in selected_points for i in [12, 14, 16]):
                                    angle_elbow_right = calculate_angle_3d(get_xyz(12), get_xyz(14), get_xyz(16))
                                    frame_data.append(angle_elbow_right)
                                else:
                                    frame_data.append(0.0) # Pad if points not selected

                                # Angle genou gauche (23, 25, 27)
                                if all(i in selected_points for i in [23, 25, 27]):
                                    angle_knee_left = calculate_angle_3d(get_xyz(23), get_xyz(25), get_xyz(27))
                                    frame_data.append(angle_knee_left)
                                else:
                                    frame_data.append(0.0) # Pad if points not selected

                                # Orientation du torse (11, 12, 23, 24)
                                if all(i in selected_points for i in [11, 12, 23, 24]):
                                    shoulder_left = np.array(get_xyz(11))
                                    shoulder_right = np.array(get_xyz(12))
                                    hip_left = np.array(get_xyz(23))
                                    hip_right = np.array(get_xyz(24))

                                    mid_shoulder = (shoulder_left + shoulder_right) / 2
                                    mid_hip = (hip_left + hip_right) / 2
                                    y_axis = np.array([0, 1, 0])

                                    # Ensure mid_shoulder and mid_hip are not identical to avoid issues
                                    if np.linalg.norm(mid_shoulder - mid_hip) > 1e-6:
                                         torso_angle = calculate_angle_3d(mid_hip, mid_shoulder, mid_shoulder + y_axis)
                                    else:
                                         torso_angle = 0.0 # Handle case where hip and shoulder are at same point
                                    frame_data.append(torso_angle)
                                else:
                                    frame_data.append(0.0) # Pad if points not selected


                            except Exception as e:
                                print(f"Error calculating angles for frame {current_frame}: {e}")
                                frame_data.extend([0.0] * 3) # Pad with 0s in case of error


                            # Distances relatives (using selected points where applicable)
                            try:
                                if all(i in selected_points for i in [14, 16]):
                                    dist_wrist_elbow_r = calculate_distance(get_xyz(14), get_xyz(16))
                                else:
                                    dist_wrist_elbow_r = 0.0 # Pad if points not selected

                                if all(i in selected_points for i in [23, 25]):
                                    dist_hip_knee_l = calculate_distance(get_xyz(23), get_xyz(25))
                                else:
                                    dist_hip_knee_l = 0.0 # Pad if points not selected

                                frame_data.extend([dist_wrist_elbow_r, dist_hip_knee_l])
                            except Exception as e:
                                print(f"Error calculating distances for frame {current_frame}: {e}")
                                frame_data.extend([0.0, 0.0]) # Pad with 0s in case of error


                            # Calcul vitesse et accélération (for selected points)
                            try:
                                if prev_positions is not None and len(prev_positions) == len(current_positions):
                                    velocities = [calculate_velocity(p, c) for p, c in zip(prev_positions, current_positions)]
                                    flat_vel = np.array(velocities).flatten()
                                    frame_data.extend(flat_vel)

                                    if prev_velocities is not None and len(prev_velocities) == len(velocities):
                                        accelerations = [calculate_acceleration(pv, cv) for pv, cv in zip(prev_velocities, velocities)]
                                        flat_acc = np.array(accelerations).flatten()
                                        frame_data.extend(flat_acc)
                                    else:
                                        # Pad if previous velocities are not available
                                        frame_data.extend([0.0] * (len(selected_points) * 3))

                                    prev_velocities = velocities
                                else:
                                    # Premier frame ou erreur de longueur
                                    # Pad velocity and acceleration with 0s
                                    frame_data.extend([0.0] * (len(selected_points) * 3 * 2))
                                    prev_velocities = None

                                prev_positions = current_positions
                            except Exception as e:
                                print(f"Error calculating dynamics for frame {current_frame}: {e}")
                                frame_data.extend([0.0] * (len(selected_points) * 3 * 2)) # Pad with 0s in case of error


                        else:
                             # Aucun landmark détecté ou safe_landmarks is empty after processing
                            # Fill frame_data with 0s to match expected_length
                            frame_data = [0.0] * expected_length
                            # Reset dynamics trackers if no landmarks detected
                            prev_positions = None
                            prev_velocities = None


                    else:
                        # Aucun landmark détecté par MediaPipe
                        # Fill frame_data with 0s to match expected_length
                        frame_data = [0.0] * expected_length
                        # Reset dynamics trackers if no landmarks detected
                        prev_positions = None
                        prev_velocities = None


                    # Normalisation de la longueur des features par frame
                    # Ensure the frame data length matches the expected length for the current exercise
                    if len(frame_data) < expected_length:
                        # This should not happen if padding logic is correct, but as a safeguard
                        print(f"Warning: Frame data length {len(frame_data)} less than expected {expected_length} for frame {current_frame}. Padding.")
                        frame_data.extend([0.0] * (expected_length - len(frame_data)))
                    elif len(frame_data) > expected_length:
                        # This indicates an error in feature calculation logic or expected_length calculation
                        print(f"Error: Frame data length {len(frame_data)} greater than expected {expected_length} for frame {current_frame}. Truncating.")
                        frame_data = frame_data[:expected_length]

                    frames.append(frame_data)

                except Exception as e:
                    print(f"Erreur inattendue lors du traitement de la frame {current_frame} pour {Path(video_path).name}: {e}")
                    # Frame de fallback avec 0s
                    frame_data = [0.0] * expected_length
                    frames.append(frame_data)
                    # Reset dynamics trackers in case of unexpected error
                    prev_positions = None
                    prev_velocities = None


            current_frame += 1

        cap.release()

        print(f"Finished processing {Path(video_path).name}. Processed {processed_frames_count} frames, detected landmarks in {landmarks_detected_count} frames.")

        # Reset des filtres pour la prochaine vidéo
        if self.use_temporal_smoothing and exercise_name in self.temporal_filters:
            for filter_obj in self.temporal_filters[exercise_name].values():
                filter_obj.reset()

        # Découpage en chunks avec validation de données
        chunks = []
        # Ensure we have enough frames to form at least one chunk
        if len(frames) >= self.max_frames:
            for i in range(0, len(frames) - self.max_frames + 1, self.max_frames):
                chunk = frames[i:i + self.max_frames]
                if len(chunk) == self.max_frames:
                    chunk_array = np.array(chunk, dtype=np.float64)  # float64 pour stabilité

                    # Validation finale du chunk
                    if not np.isnan(chunk_array).any() and not np.isinf(chunk_array).any():
                         # Before appending, ensure the chunk's feature dimension matches the expected_length
                         if chunk_array.shape[1] == expected_length:
                             chunks.append(chunk_array)
                         else:
                             print(f"Skipping chunk due to incorrect feature dimension ({chunk_array.shape[1]} vs {expected_length}) in video {Path(video_path).name}")

                    else:
                        print(f"Skipping chunk due to NaN or Inf values in video {Path(video_path).name}")
        else:
             print(f"Not enough frames ({len(frames)}) to form a chunk of size {self.max_frames} for video {Path(video_path).name}")


        print(f"Generated {len(chunks)} chunks for video {Path(video_path).name}")
        print("--- End of video processing ---")


        return chunks

    def validate_output_quality(self, chunks, exercise_name):
        """Valide la qualité des données générées"""
        if not chunks:
            return {"status": "error", "message": "Aucun chunk généré"}

        metrics = {
            "num_chunks": len(chunks),
            "chunk_shapes": [chunk.shape for chunk in chunks],
            "nan_count": sum(np.isnan(chunk).sum() for chunk in chunks),
            "inf_count": sum(np.isinf(chunk).sum() for chunk in chunks),
            "zero_frames": sum((chunk == 0).all(axis=1).sum() for chunk in chunks),
            "data_variance": np.mean([np.var(chunk) for chunk in chunks]) if chunks else 0 # Handle empty chunks list
        }

        # Validation de cohérence
        selected_points = exercises_points[exercise_name]
        expected_base_features = len(selected_points) * 3
        # This must match the feature calculation in process_video_enhanced
        expected_feature_length = expected_base_features + 3 + 2 + (expected_base_features * 2)


        all_shapes_correct = all(chunk.shape == (self.max_frames, expected_feature_length) for chunk in chunks)

        status = "success"
        if not all_shapes_correct or metrics["nan_count"] > 0 or metrics["inf_count"] > 0:
            status = "warning"
        if metrics["data_variance"] < 1e-8:  # Données trop static_image_modes
            status = "warning"

        return {
            "status": status,
            "metrics": metrics,
            "shapes_correct": all_shapes_correct
        }

class EnhancedVideoLandmarksDataset(Dataset):
    """Dataset amélioré avec preprocesseur enhanced"""

    def __init__(self, root_dir, mode='train', max_per_class=None): # Changed max_per_class default to None
        self.root_dir = Path(root_dir)
        self.mode = mode
        self.max_per_class = max_per_class

        self.preprocessor = EnhancedExercisePreprocessor(
            use_physics_validation=True,
            use_temporal_smoothing=True
        )

        # Keep track of exercises that were successfully loaded and processed
        self.present_exercises = []
        self.samples = []
        self.labels = []
        self.groups = []
        self.exercise_names = []

        # Mappings will be built dynamically based on loaded data
        self.exercise_to_idx = {}
        self.idx_to_exercise = {}
        self.exercise_to_group = {}
        self.group_to_idx = {}
        self.idx_to_group = {}
        self.max_feature_dim = 0 # Will be calculated after loading

        self._load_samples_and_build_mappings()


        if self.samples and self.preprocessor.use_physics_validation:
            self._validate_dataset_quality()


    def __len__(self):
        return len(self.samples)

    def _load_samples_and_build_mappings(self):
        """Loads samples and builds mappings based on available data and defined groups/points."""
        all_samples = []
        all_exercise_names = []
        loaded_count = 0
        skipped_count = 0

        # Determine which exercises have data and are in the defined exercises_points and group_mapping
        exercises_with_data = []
        if os.path.isdir(self.root_dir):
             for exercise_name in os.listdir(self.root_dir):
                 exercise_dir = self.root_dir / exercise_name
                 if exercise_dir.is_dir() and exercise_name in exercises_points:
                     # Check if the exercise is in any defined group
                     is_in_a_group = any(exercise_name in exercises for exercises in group_mapping.values())
                     if is_in_a_group:
                          exercises_with_data.append(exercise_name)

        self.present_exercises = sorted(exercises_with_data)
        print(f"Found {len(self.present_exercises)} present exercises with data and group mapping.")


        # Build dynamic mappings only for present exercises and groups
        self.exercise_to_idx = {ex: i for i, ex in enumerate(self.present_exercises)}
        self.idx_to_exercise = {i: ex for ex, i in self.exercise_to_idx.items()}
        print(f"exercise_to_idx (dynamic): {self.exercise_to_idx}")

        self.exercise_to_group = {}
        for group_name, exercises in group_mapping.items():
            for ex in exercises:
                if ex in self.exercise_to_idx:
                    self.exercise_to_group[ex] = group_name
        print(f"exercise_to_group (dynamic): {self.exercise_to_group}")

        present_groups = sorted(list(set(self.exercise_to_group.values())))
        self.group_to_idx = {g: i for i, g in enumerate(present_groups)}
        self.idx_to_group = {i: g for g, i in self.group_to_idx.items()}
        print(f"group_to_idx (dynamic): {self.group_to_idx}")


        # Load sample paths and labels
        for exercise_name in self.present_exercises:
            exercise_dir = self.root_dir / exercise_name
            npy_files = sorted(list(exercise_dir.glob('*.npy')))

            # Apply max_per_class limit if specified
            if self.max_per_class is not None:
                npy_files = npy_files[:self.max_per_class]

            for npy_file in npy_files:
                self.samples.append(str(npy_file)) # Store as string
                self.labels.append(self.exercise_to_idx[exercise_name])
                self.groups.append(self.group_to_idx[self.exercise_to_group[exercise_name]])
                self.exercise_names.append(exercise_name)
                loaded_count += 1


        print(f"Finished collecting sample paths. Found {loaded_count} potential samples.")

        # Calculate max_feature_dim after identifying present exercises
        self.max_feature_dim = self._calculate_max_feature_dim(self.present_exercises)
        print(f"Calculated max_feature_dim: {self.max_feature_dim}")


    def _calculate_max_feature_dim(self, present_exercises):
        """Calculates the maximum feature dimension across the present exercises."""
        max_dim = 0
        for exercise_name in present_exercises:
            if exercise_name in exercises_points: # Use the global exercises_points here
                num_points = len(exercises_points[exercise_name])
                # Coordonnées 3D + Angles + Distances + Vitesses + Accélérations
                # This must match the feature calculation in EnhancedExercisePreprocessor
                base_features = num_points * 3
                angles = 3 # Fixed to 3 based on preprocessor logic
                distances = 2 # Fixed to 2 based on preprocessor logic
                dynamics = base_features * 2
                current_dim = base_features + angles + distances + dynamics
                max_dim = max(max_dim, current_dim)
        # If no exercises are present, return a default or handle appropriately
        return max_dim if max_dim > 0 else 0


    def _validate_dataset_quality(self):
      """Valide les échantillons chargés (placeholder simple)."""
      # Here, we validate after loading paths, before returning items
      # This validation checks the integrity and basic quality of the .npy files
      valid_indices = []
      corrupted_count = 0
      quality_issue_count = 0

      print(f"Starting dataset quality validation on {len(self.samples)} samples...")

      for i, path in enumerate(self.samples):
          try:
              arr = np.load(path, mmap_mode='r')

              # Check shape consistency (features dimension must match the expected for this exercise)
              exercise_name = self.exercise_names[i]
              expected_feature_dim_for_exercise = self._calculate_expected_features(exercise_name) # Calculate expected dim for this specific exercise


              if arr.shape[0] != self.preprocessor.max_frames or arr.shape[1] != expected_feature_dim_for_exercise:
                  print(f"Warning: {Path(path).name} has incorrect shape {arr.shape}. Expected ({self.preprocessor.max_frames}, {expected_feature_dim_for_exercise}). Skipping.")
                  corrupted_count += 1
                  continue

              # Check for NaN/Inf values
              if np.isnan(arr).any() or np.isinf(arr).any():
                  print(f"Warning: {Path(path).name} contains NaN or Inf values. Skipping.")
                  quality_issue_count += 1
                  continue

              # Check for excessive zero frames (optional, based on expected data distribution)
              # zero_frame_ratio = (arr == 0).all(axis=1).sum() / arr.shape[0]
              # if zero_frame_ratio > 0.8: # Example threshold
              #     print(f"Warning: {Path(path).name} has a high ratio of zero frames ({zero_frame_ratio:.1%}). Skipping.")
              #     quality_issue_count += 1
              #     continue

              # Check for very low variance (static data)
              variance = np.var(arr)
              if variance < 1e-8: # Example threshold
                   print(f"Warning: {Path(path).name} has very low variance ({variance:.2e}). Data might be static. Skipping.")
                   quality_issue_count += 1
                   continue


              valid_indices.append(i)

          except Exception as e:
              print(f"Error loading or validating {Path(path).name}: {e}. Skipping.")
              corrupted_count += 1

      # Filter the samples, labels, groups, and exercise_names lists based on valid indices
      self.samples = [self.samples[i] for i in valid_indices]
      self.labels = [self.labels[i] for i in valid_indices]
      self.groups = [self.groups[i] for i in valid_indices]
      self.exercise_names = [self.exercise_names[i] for i in valid_indices]


      print(f"Dataset quality validation complete. {len(self)} valid samples, {corrupted_count} corrupted, {quality_issue_count} with quality issues.")


    def _calculate_expected_features(self, exercise_name):
        """Calculates the expected feature dimension for a given exercise based on the preprocessor logic."""
        if exercise_name not in exercises_points: # Use the global exercises_points
            # Fallback or error handling if exercise is not in the points mapping
            print(f"Warning: Exercise '{exercise_name}' not found in exercises_points during validation.")
            # Return the max dimension as a safe fallback, or raise an error
            return self.max_feature_dim # Use max_feature_dim calculated earlier

        num_points = len(exercises_points[exercise_name]) # Use the global exercises_points
        base_features = num_points * 3

        # Features from enhanced preprocessor (Angles, Distances, Velocities, Accelerations)
        # This calculation MUST match the feature generation in process_video_enhanced
        angles = 3  # coude, genou, torse
        distances = 2  # distances relatives
        dynamics = base_features * 2  # vitesses + accélérations (3D * 2 for vel+acc)

        total_features = base_features + angles + distances + dynamics
        return total_features


    def augment_sequence(self, seq):
      """Enhanced data augmentation"""
      # Controlled Jitter
      if random.random() < 0.5 and seq.numel() > 0: # Added check for empty tensor
          noise_level = 0.02 * torch.std(seq)
          seq += torch.randn_like(seq) * noise_level

      # Adaptive Scaling
      if random.random() < 0.5 and seq.numel() > 0: # Added check for empty tensor
          scale = random.uniform(0.95, 1.05)
          seq *= scale

      # Selective Masking with PyTorch
      if random.random() < 0.3 and seq.numel() > 0: # Added check for empty tensor
          mask = (torch.rand_like(seq) > 0.15).float()
          mean_val = torch.mean(seq) if seq.numel() > 0 else 0.0 # Handle empty tensor mean
          seq = seq * mask + (1 - mask) * mean_val

      # Temporal Permutation with PyTorch
      if random.random() < 0.2 and seq.size(0) > 5: # Ensure enough frames for splitting
          seq_len = seq.size(0)
          try:
              split_indices = sorted(random.sample(range(1, seq_len), random.randint(1, min(3, seq_len - 1))))
              split_sizes = [split_indices[0]]
              for i in range(len(split_indices) - 1):
                  split_sizes.append(split_indices[i+1] - split_indices[i])
              split_sizes.append(seq_len - split_indices[-1])
              segments = torch.split(seq, split_sizes)

              indices = torch.randperm(len(segments))
              seq = torch.cat([segments[i] for i in indices], dim=0)[:seq_len] # Ensure length is preserved
          except ValueError as e:
               print(f"Warning: Temporal permutation failed for a sample. {e}") # Log error

      return seq

    def __getitem__(self, idx):
        try:
            # Load the numpy array (already validated in _validate_dataset_quality)
            seq_np = np.load(self.samples[idx], mmap_mode='r', allow_pickle=False)
            seq = torch.from_numpy(seq_np).to(torch.float32) # Ensure float32 and move to device


            # Standardize sequence length and feature dimension using the global max_feature_dim
            seq = self._standardize_sequence_enhanced(seq, self.max_feature_dim)


            if self.augment and self.mode == 'train': # Only augment training data
                seq = self.augment_sequence(seq)

            return (
                seq, # Already on device from _standardize_sequence_enhanced
                torch.tensor(self.labels[idx], dtype=torch.long).to(device),
                torch.tensor(self.groups[idx], dtype=torch.long).to(device)
            )

        except Exception as e:
            print(f"❌ Error loading or processing {self.samples[idx]} in __getitem__: {str(e)}")
            return None # Return None for problematic samples

    def _standardize_sequence_enhanced(self, seq, target_feature_dim):
        """Adaptive standardization to a target feature dimension and max sequence length."""
        # Temporal adjustment
        if seq.shape[0] > self.max_seq_len:
            seq = seq[:self.max_seq_len]
        elif seq.shape[0] < self.max_seq_len:
            padding = self.max_seq_len - seq.shape[0]
            seq = torch.nn.functional.pad(seq, (0, 0, 0, padding), mode='constant', value=0) # Pad rows

        # Feature adjustment
        if seq.shape[1] < target_feature_dim:
            padding = target_feature_dim - seq.shape[1]
            seq = torch.nn.functional.pad(seq, (0, padding), mode='constant', value=0) # Pad columns
        elif seq.shape[1] > target_feature_dim:
            seq = seq[:, :target_feature_dim]


        return seq.to(device) # Move to device after padding/truncating


# Fonctions utilitaires du pipeline
def pretraiter_videos_enhanced(config):
    """Pipeline de prétraitement amélioré avec validation de qualité et gestion d'erreurs détaillée"""
    preprocessor = EnhancedExercisePreprocessor(
        max_frames=config['max_frames'],
        step=config.get('step', 2),
        use_physics_validation=config.get('use_physics_validation', True),
        use_temporal_smoothing=config.get('use_temporal_smoothing', True)
    )

    # Statistiques globales
    total_videos_attempted = 0
    total_videos_processed = 0
    total_chunks_generated = 0
    quality_reports = {}
    error_counts = {} # To track specific error types

    for exercise in config['classes_a_traiter']:
        input_dir = Path(config['input_dir']) / exercise
        output_dir = Path(config['output_dir']) / exercise
        output_dir.mkdir(parents=True, exist_ok=True)

        if not input_dir.exists():
            print(f"⚠️ Répertoire source manquant pour {exercise}: {input_dir}")
            continue

        # Extensions vidéos supportées
        video_formats = ['*.mp4', '*.avi', '*.mov', '*.mkv', '*.webm']
        video_files = []
        for ext in video_formats:
            video_files.extend(input_dir.glob(ext))

        total_videos_attempted += len(video_files)

        exercise_chunks = 0
        exercise_processed = 0

        print(f"\n🎯 Traitement de {exercise} ({len(video_files)} vidéos)")

        for video_file in tqdm(video_files, desc=f"Processing {exercise}"):
            try:
                prefix = video_file.stem

                # Vérification si déjà traité
                existing_parts = list(output_dir.glob(f"{prefix}_part*.npy"))
                if existing_parts and config.get('skip_existing', True):
                    # Validation rapide des fichiers existants
                    all_valid = True
                    expected_base_features = len(exercises_points.get(exercise, [])) * 3
                    # Must match the feature calculation in process_video_enhanced
                    expected_feature_length = expected_base_features + 3 + 2 + (expected_base_features * 2)

                    for part_file in existing_parts:
                        try:
                            arr = np.load(part_file, mmap_mode='r') # Use mmap_mode for efficiency
                            if arr.shape[0] != config['max_frames'] or arr.shape[1] != expected_feature_length or np.isnan(arr).any() or np.isinf(arr).any():
                                all_valid = False
                                break
                        except Exception as e:
                            print(f"Error validating existing file {part_file}: {e}")
                            all_valid = False
                            break

                    if all_valid:
                        exercise_chunks += len(existing_parts)
                        exercise_processed += 1
                        continue # Skip to next video if valid existing files found


                # Traitement avec preprocesseur amélioré
                # This call now includes more detailed logging within if errors occur
                chunks = preprocessor.process_video_enhanced(str(video_file), exercise)

                if not chunks:
                     print(f"⚠️ Aucune chunk générée pour {video_file.name} ({exercise})")
                     # Log specific reasons if available from process_video_enhanced, or a general message
                     continue # Skip to next video if no chunks are generated

                # Validation de qualité sur les chunks générés
                quality_report = preprocessor.validate_output_quality(chunks, exercise)

                if quality_report["status"] == "error":
                    print(f"❌ Erreur qualité majeure pour {video_file.name}: {quality_report['message']}. Skipping.")
                    # Optionally delete the incomplete or bad quality files
                    for existing_part in output_dir.glob(f"{prefix}_part*.npy"):
                         try:
                             os.remove(existing_part)
                             print(f"🗑️ Supprimé fichier de qualité médiocre: {existing_part.name}")
                         except:
                             pass
                    continue


                # Sauvegarde des chunks
                for i, chunk in enumerate(chunks):
                    npy_path = output_dir / f"{prefix}_part{i+1}.npy"
                    # Ensure chunk is float32 before saving for consistency, although numpy handles it
                    np.save(npy_path, chunk.astype(np.float32))

                exercise_chunks += len(chunks)
                exercise_processed += 1

                # Rapport de qualité pour debug (warnings)
                if quality_report["status"] == "warning":
                    print(f"⚠️ {video_file.name}: Qualité suboptimale détectée. {quality_report['metrics']}")


            except Exception as e:
                print(f"❌ Erreur inattendue lors du traitement de {video_file.name} ({exercise}): {str(e)}")
                # Increment a general error count for this exercise
                error_counts[exercise] = error_counts.get(exercise, 0) + 1
                continue # Continue to the next video


        total_videos_processed += exercise_processed
        total_chunks_generated += exercise_chunks
        quality_reports[exercise] = {
            "processed_videos": exercise_processed,
            "generated_chunks": exercise_chunks,
            "errors": error_counts.get(exercise, 0)
        }

        print(f"✅ {exercise}: {exercise_processed} vidéos traitées → {exercise_chunks} chunks générés")

    # Rapport final
    print(f"\n📊 RAPPORT FINAL DE PRÉTRAITEMENT:")
    print(f"   Total vidéos tentées: {total_videos_attempted}")
    print(f"   Total vidéos traitées avec succès: {total_videos_processed}")
    print(f"   Total chunks générés: {total_chunks_generated}")
    print(f"   Qualité améliorée activée: {'✅' if config.get('use_physics_validation', True) else '❌'}")
    print(f"   Lissage temporel activé: {'✅' if config.get('use_temporal_smoothing', True) else '❌'}")
    if error_counts:
         print("   Récapitulatif des erreurs par exercice:")
         for ex, count in error_counts.items():
             print(f"     - {ex}: {count} erreurs inattendues")
    else:
         print("   Aucune erreur inattendue signalée.")


def verifier_coherence_enhanced(config):
    """Vérification améliorée avec métriques de qualité physique"""
    print("🔍 Vérification de cohérence avancée des fichiers prétraités...")

    # We no longer need PhysicsValidator instance here, as validation is done during preprocessing
    # This function primarily checks file integrity and shape consistency

    total_files_found = 0
    corrupted_files = 0
    quality_issues = 0 # This now refers to issues detected during preprocessing and potentially logged here

    for exercise in config['classes_a_traiter']:
        output_dir = Path(config['output_dir']) / exercise

        if not output_dir.exists():
            print(f"⚠️ Dossier manquant pour la vérification: {output_dir}")
            continue

        # Recalculate expected feature length based on exercises_points
        selected_points = exercises_points.get(exercise, [])
        if not selected_points:
            print(f"Warning: Exercise '{exercise}' not found in exercises_points during verification. Skipping.")
            continue

        expected_base_features = len(selected_points) * 3
        # This must match the feature calculation in preprocessor.process_video_enhanced
        expected_feature_length = expected_base_features + 3 + 2 + (expected_base_features * 2)


        print(f"\n📂 Vérification {exercise}...")

        npy_files = list(output_dir.glob('*.npy'))
        if not npy_files:
            print(f"  Aucun fichier .npy trouvé pour {exercise} dans {output_dir}")
            continue


        for npy_file in tqdm(npy_files, desc=f"Checking {exercise}"):
            total_files_found += 1

            try:
                arr = np.load(npy_file, mmap_mode='r') # Use mmap_mode for efficiency

                # Vérifications de base: forme et valeurs non finies
                if arr.shape[0] != config['max_frames'] or arr.shape[1] != expected_feature_length:
                     print(f"❌ Fichier corrompu: {npy_file.name} - Forme incorrecte {arr.shape} vs attendu ({config['max_frames']}, {expected_feature_length}).")
                     corrupted_files += 1
                     try:
                         os.remove(npy_file)
                         print(f"🗑️ Fichier supprimé: {npy_file.name}")
                     except:
                         pass
                     continue # Move to next file

                if np.isnan(arr).any() or np.isinf(arr).any():
                     print(f"❌ Fichier corrompu: {npy_file.name} - Contient NaN ou Inf.")
                     corrupted_files += 1
                     try:
                         os.remove(npy_file)
                         print(f"🗑️ Fichier supprimé: {npy_file.name}")
                     except:
                         pass
                     continue # Move to next file


                # Additional quality checks (optional, based on expected data distribution)
                # These were already done during preprocessing, but can be re-checked
                # zero_frame_ratio = (arr == 0).all(axis=1).sum() / arr.shape[0]
                # if zero_frame_ratio > 0.5:
                #     # This might indicate a preprocessing issue, but the file is not 'corrupted' in format
                #     # You could log this as a quality issue rather than corrupted.
                #     # print(f"⚠️ {npy_file.name}: {zero_frame_ratio:.1%} frames nulles")
                #     # quality_issues += 1
                #     pass # Decide how to handle this based on your needs

                # variance = np.var(arr)
                # if variance < 1e-8:
                #     # print(f"⚠️ {npy_file.name}: Très faible variance ({variance:.2e}).")
                #     # quality_issues += 1
                #     pass # Decide how to handle this


            except Exception as e:
                corrupted_files += 1
                print(f"❌ Fichier corrompu: {npy_file.name} - Erreur de chargement ou de vérification: {e}")
                try:
                    os.remove(npy_file)
                    print(f"🗑️ Fichier supprimé: {npy_file.name}")
                except:
                    pass

    print(f"\n📊 RAPPORT DE VÉRIFICATION DES FICHIERS:")
    print(f"   Fichiers .npy trouvés: {total_files_found}")
    print(f"   Fichiers corrompus ou de forme incorrecte: {corrupted_files}")
    # print(f"   Problèmes de qualité signalés (peuvent inclure les corrompus): {quality_issues}") # If tracking separately
    print(f"   Fichiers valides (forme correcte, sans NaN/Inf): {total_files_found - corrupted_files}")
    print(f"   Taux de succès de vérification: {((total_files_found - corrupted_files) / max(total_files_found, 1)):.1%}")
import os


def quick_start_enhanced():
    input_dir = '/content/drive/MyDrive/filtered_datasets'
    config = {
        'input_dir': input_dir,
        'output_dir': '/content/drive/MyDrive/preprocessed_videos_enhanced',
        'max_frames': 50,
        'step': 2,
        'use_physics_validation': True,
        'use_temporal_smoothing': True,
        'skip_existing': True,
        'classes_a_traiter': [
            d for d in os.listdir(input_dir)
            if os.path.isdir(os.path.join(input_dir, d))
        ]
    }
    print("🚀 Démarrage du prétraitement amélioré...")
    print(f"   Répertoire d'entrée: {config['input_dir']}")
    print(f"   Répertoire de sortie: {config['output_dir']}")
    print(f"   Classes à traiter: {config['classes_a_traiter']}")
    print(f"   Contraintes physiques: {'✅' if config['use_physics_validation'] else '❌'}")
    print(f"   Lissage temporel: {'✅' if config['use_temporal_smoothing'] else '❌'}")
    print(f"   Max frames per chunk: {config['max_frames']}")

    # Exécution du pipeline amélioré
    pretraiter_videos_enhanced(config)

    # Vérification de cohérence avancée
    verifier_coherence_enhanced(config)

    print("\n🎉 Prétraitement amélioré terminé!")

if __name__ == "__main__":
    quick_start_enhanced()

🚀 Démarrage du prétraitement amélioré...
   Répertoire d'entrée: /content/drive/MyDrive/filtered_datasets
   Répertoire de sortie: /content/drive/MyDrive/preprocessed_videos_enhanced
   Classes à traiter: ['plank', 'incline bench press', 'lat pulldown', 'bench press', 'decline bench press', 'tricep pushdown', 'romanian deadlift', 'leg extension', 'lateral raise', 'push-up', 'chest fly machine', 'shoulder press', 'russian twist', 'squat', 'leg raises', 'pull up', 'barbell biceps curl', 'deadlift', 't bar row', 'hip thrust', 'hammer curl', 'tricep dips']
   Contraintes physiques: ✅
   Lissage temporel: ✅
   Max frames per chunk: 50

🎯 Traitement de plank (114 vidéos)


Processing plank:   0%|          | 0/114 [00:00<?, ?it/s]

--- Processing video: plank_2_f62d9bff.mp4 for exercise plank ---


Processing plank:   1%|          | 1/114 [00:04<07:44,  4.11s/it]

Finished processing plank_2_f62d9bff.mp4. Processed 39 frames, detected landmarks in 39 frames.
Not enough frames (39) to form a chunk of size 50 for video plank_2_f62d9bff.mp4
Generated 0 chunks for video plank_2_f62d9bff.mp4
--- End of video processing ---
⚠️ Aucune chunk générée pour plank_2_f62d9bff.mp4 (plank)


...


📂 Vérification hammer curl...


Checking hammer curl: 100%|██████████| 135/135 [00:01<00:00, 97.62it/s] 



📂 Vérification tricep dips...


Checking tricep dips: 100%|██████████| 144/144 [00:01<00:00, 95.60it/s] 


📊 RAPPORT DE VÉRIFICATION DES FICHIERS:
   Fichiers .npy trouvés: 3213
   Fichiers corrompus ou de forme incorrecte: 0
   Fichiers valides (forme correcte, sans NaN/Inf): 3213
   Taux de succès de vérification: 100.0%

🎉 Prétraitement amélioré terminé!


In [3]:
landmarks_mp = {
    0: "nez",
    11: "epaule gauche",
    12: "epaule droite",
    13: "coude gauche",
    14: "coude droit",
    15: "poignet gauche",
    16: "poignet droit",
    23: "hanche gauche",
    24: "hanche droite",
    25: "genou gauche",
    26: "genou droit",
    27: "cheville gauche",
    28: "cheville droite"
}

# Configuration MediaPipe optimisée
MP_CONFIG = {
    'static_image_mode': False,
    'model_complexity': 2,
    'min_detection_confidence': 0.5,
    'min_tracking_confidence': 0.5,
    'enable_segmentation': True,
    'smooth_landmarks': True,
    'smooth_segmentation': False
}

# Points clés par exercice (configuration originale préservée)

exercises_points = {
    "t bar row":            [0, 11, 12, 13, 14, 15, 16, 23, 24],
    "pull up":              [0, 11, 12, 13, 14, 15, 16],
    "hammer curl":          [11, 12, 13, 14, 15, 16],
    "decline bench press":  [11, 12, 13, 14, 15, 16, 23, 24],
    "tricep pushdown":      [11, 12, 13, 14, 15, 16],
    "chest fly machine":    [11, 12, 13, 14, 15, 16],
    "squat":                [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],  # +1 bassin
    "bench press":          [11, 12, 13, 14, 15, 16, 23, 24],
    "push-up":              [0, 11, 12, 13, 14, 15, 16, 23, 24, 27, 28],
    "deadlift":             [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],  # +1 bassin
    "leg raises":           [0, 23, 24, 25, 26, 27, 28],
    "russian twist":        [0, 1, 11, 12, 15, 16, 23, 24],  # +1 bassin
    "barbell biceps curl":  [11, 12, 13, 14, 15, 16],
    "lateral raise":        [11, 12, 13, 14, 15, 16],
    "hip thrust":           [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],  # +1 bassin
    "lat pulldown":         [11, 12, 13, 14, 15, 16],
    "plank":                [0, 1, 11, 12, 23, 24, 27, 28],  # +1 bassin
    "incline bench press":  [11, 12, 13, 14, 15, 16, 23, 24],
    "tricep dips":          [11, 12, 13, 14, 15, 16],
    "leg extension":        [23, 24, 25, 26, 27, 28],
    "romanian deadlift":    [0, 1, 11, 12, 23, 24, 25, 26, 27, 28],  # +1 bassin
    "shoulder press":       [0, 11, 12, 13, 14, 15, 16, 23, 24]
}

group_mapping = {
    "horizontal pushes": [
        "bench press",
        "incline bench press",
        "push-up"
    ],
    "vertical pushes": [
        "shoulder press",
        "tricep dips",
        "tricep pushdown",
        "lateral raise"  # ajoutÃ© ici
    ],
    "horizontal pulls": [
        "t bar row",
        "barbell biceps curl",
        "hammer curl"
    ],
    "vertical pulls": [
        "pull up",
        "lat pulldown",
        "chest fly machine"
    ],
    "hip dominant": [
        "deadlift",
        "romanian deadlift",
        "hip thrust"
    ],
    "quad dominant": [
        "squat",
        "leg extension",
        "decline bench press"
    ],
    "core training": [
        "plank",
        "leg raises",
        "russian twist"
    ]
}

nombre_de_groupes = len(group_mapping)

print("Nombre de groupes :", nombre_de_groupes)
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"""
def augment_landmarks(landmarks):
    if np.random.rand() > 0.5:
        landmarks = flip_landmarks(landmarks)
    if np.random.rand() > 0.5:
        landmarks = add_noise(landmarks, noise_level=0.02)
    if np.random.rand() > 0.5:
        landmarks = random_scale(landmarks, scale_range=(0.95, 1.05))
    return augment_landmarks
"""
import os
import numpy as np
import torch
from torch.utils.data import Dataset
import random
from pathlib import Path




class EnhancedExerciseHierarchicalDataset(Dataset):
    def __init__(self, data_dir, exercices_points, grouped_sports, max_seq_len=45,
                 augment=False, validate_physics=True): # Removed base_feature_dim here
        self.augment = augment
        self.max_seq_len = max_seq_len
        self.validate_physics = validate_physics
        self.data_dir = data_dir
        self.exercise_names = []

        # Cache des dimensions par exercice
        self.feature_cache = {}
        self.exercises_points = exercices_points
        self.grouped_sports = grouped_sports # Keep grouped_sports for mapping

        # Construction des mappings et chargement avec validation (dynamic)
        self._build_mappings_dynamic()

        # Determine the maximum feature dimension across the dynamically loaded exercises
        self.max_feature_dim = self._calculate_max_feature_dim(self.present_exercises)

        self._load_samples_enhanced()

        if validate_physics:
            self._validate_dataset_quality()

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.samples)


    def _calculate_max_feature_dim(self, present_exercises):
        """Calculates the maximum feature dimension across the present exercises."""
        max_dim = 0
        for exercise_name in present_exercises:
            if exercise_name in self.exercises_points:
                num_points = len(self.exercises_points[exercise_name])
                # CoordonnÃ©es 3D + Angles + Distances + Vitesses + AccÃ©lÃ©rations
                current_dim = num_points * 3 + 3 + 2 + num_points * 3 + num_points * 3
                max_dim = max(max_dim, current_dim)
        # If no exercises are present, return a default or handle appropriately
        return max_dim if max_dim > 0 else 0


    def _validate_dataset_quality(self):
      """Valide les échantillons chargés (placeholder simple)."""
      # Ici, tu peux ajouter des vérifications avancées, par par exemple :
      valid_samples = []
      valid_labels = []
      valid_groups = []
      valid_exercise_names = []
      for i, path in enumerate(self.samples):
          try:
              arr = np.load(path, mmap_mode='r')
              # Exemple : on vérifie qu'il n'y a pas de NaN
              if not np.isnan(arr).any() and not np.isinf(arr).any():
                  valid_samples.append(path)
                  valid_labels.append(self.labels[i])
                  valid_groups.append(self.groups[i])
                  valid_exercise_names.append(self.exercise_names[i])
          except Exception as e:
              print(f"Fichier corrompu ou illisible: {path} ({e})")
      print(f"Après validation: {len(valid_samples)} séquences valides / {len(self.samples)}")
      self.samples = valid_samples
      self.labels = valid_labels
      self.groups = valid_groups
      self.exercise_names = valid_exercise_names


    def _build_mappings_dynamic(self):
        """Construit les mappings exercice <-> groupe dynamiquement basés sur les fichiers prétraités"""
        self.present_exercises = []
        print(f"Scanning directory for preprocessed data: {self.data_dir}")
        if not os.path.isdir(self.data_dir):
            print(f"Data directory not found: {self.data_dir}")
            self.exercise_to_idx = {}
            self.idx_to_exercise = {}
            self.exercise_to_group = {}
            self.group_to_idx = {}
            self.idx_to_group = {}
            return

        # Determine which exercises have preprocessed data and are in a defined group
        exercises_with_data_and_group = []
        for exercise_name in self.exercises_points:
            ex_dir = os.path.join(self.data_dir, exercise_name)
            if os.path.isdir(ex_dir):
                # Check if there is at least one .npy file in the directory
                npy_files = list(Path(ex_dir).glob('*.npy'))
                # Check if the exercise is in any defined group
                is_in_a_group = any(exercise_name in exercises for exercises in self.grouped_sports.values())

                if npy_files and is_in_a_group:
                    exercises_with_data_and_group.append(exercise_name)

        self.present_exercises = sorted(exercises_with_data_and_group)

        print(f"Found {len(self.present_exercises)} present exercises with data and group mapping.")

        # Mapping exercice -> index (only for present exercises)
        self.exercise_to_idx = {ex: i for i, ex in enumerate(self.present_exercises)}
        self.idx_to_exercise = {i: ex for ex, i in self.exercise_to_idx.items()}
        print(f"exercise_to_idx (dynamic): {self.exercise_to_idx}")
        # print(f"idx_to_exercise (dynamic): {self.idx_to_exercise}") # Avoid printing potentially long dict

        # Mapping exercice -> groupe (with verification against present exercises)
        self.exercise_to_group = {}
        for group_name, exercises in self.grouped_sports.items():
            for ex in exercises:
                if ex in self.exercise_to_idx: # Check if exercise is present and mapped
                    self.exercise_to_group[ex] = group_name
        print(f"exercise_to_group (dynamic): {self.exercise_to_group}")

        # Mapping groupe -> index (only for groups with present exercises)
        present_groups = sorted(list(set(self.exercise_to_group.values())))
        self.group_to_idx = {g: i for i, g in enumerate(present_groups)}
        self.idx_to_group = {i: g for g, i in self.group_to_idx.items()}
        print(f"group_to_idx (dynamic): {self.group_to_idx}")
        # print(f"idx_to_group (dynamic): {self.idx_to_group}") # Avoid printing potentially long dict


    def _load_samples_enhanced(self):
        """Charge les échantillons avec validation des chemins"""
        self.samples = []
        self.labels = []
        self.groups = []
        self.exercise_names = []
        loaded_count = 0
        skipped_count = 0

        if not self.present_exercises:
            print("No present exercises found with data and group. Skipping sample loading.")
            return

        for ex in self.present_exercises:
            ex_dir = os.path.join(self.data_dir, ex)
            # This directory should exist based on _build_mappings_dynamic, but double-check
            if not os.path.isdir(ex_dir):
                print(f"Warning: Directory unexpectedly missing for present exercise: {ex_dir}")
                continue

            try:
                for fname in os.listdir(ex_dir):
                    if fname.endswith('.npy'):
                        path = os.path.join(ex_dir, fname)
                        if not os.path.exists(path):
                            print(f"Fichier manquant: {path}")
                            skipped_count += 1
                            continue

                        # Basic check for file size to quickly filter out potentially empty files
                        if os.path.getsize(path) < 100: # Arbitrary small threshold
                            print(f"Skipping potentially empty file: {path}")
                            skipped_count += 1
                            continue

                        self.samples.append(path)
                        # Use the dynamically created exercise_to_idx
                        # Ensure the exercise is still in exercise_to_idx (should be based on _build_mappings)
                        if ex in self.exercise_to_idx:
                             self.labels.append(self.exercise_to_idx[ex])
                        else:
                            # This case should ideally not be reached with the updated _build_mappings
                            print(f"Warning: Exercise {ex} in samples but not in exercise_to_idx. Skipping sample.")
                            skipped_count += 1
                            self.samples.pop() # Remove the appended path
                            continue


                        # Use the dynamically created group_to_idx and exercise_to_group
                        # Ensure the exercise is still in exercise_to_group (should be based on _build_mappings)
                        if ex in self.exercise_to_group:
                            self.groups.append(self.group_to_idx[self.exercise_to_group[ex]])
                            self.exercise_names.append(ex)
                            loaded_count += 1
                        else:
                            # This case should ideally not be reached with the updated _build_mappings
                            print(f"Warning: Exercise {ex} in samples but not in exercise_to_group. Skipping sample.")
                            # Remove the appended path and label as well
                            self.samples.pop()
                            self.labels.pop()
                            skipped_count += 1


            except Exception as e:
                print(f"Error processing directory {ex_dir}: {e}")
                continue

        print(f"Finished loading samples. Loaded {loaded_count} samples, skipped {skipped_count}.")


    def _calculate_expected_features(self, exercise_name):
        """Calculates the expected feature dimension for a given exercise."""
        if exercise_name not in self.exercises_points:
            # Fallback or error handling if exercise is not in the points mapping
            print(f"Warning: Exercise '{exercise_name}' not found in exercises_points.")
            # Return a default or the max dimension
            return self.max_feature_dim # Use max_feature_dim as a safe fallback


        num_points = len(self.exercises_points[exercise_name])
        base_features = num_points * 3

        # Features du prétraitement amélioré (Angles, Distances, Vitesses, Accélérations)
        # Make sure this calculation matches the actual feature generation in preprocessor
        angles = 3  # coude, genou, torse
        distances = 2  # distances relatives
        dynamics = base_features * 2  # vitesses + accélérations (3D * 2 for vel+acc)

        total_features = base_features + angles + distances + dynamics
        return total_features

    def _extract_exercise_name(self, file_path):
        """Extrait le nom d'exercice depuis le chemin de fichier"""
        path_parts = os.path.normpath(file_path).split(os.sep)
        for part in path_parts:
            if part in self.exercises_points:
                return part
        return "unknown"

    def _validate_sample_quality(self, seq, exercise_name):
        """Valide la qualité d'un échantillon selon les critères physiques"""
        if seq.shape[0] == 0:
            return False

        # Vérification NaN/Inf
        if np.isnan(seq).any() or np.isinf(seq).any():
            return False

        # Vérification variance
        variance = np.var(seq, axis=0)
        static_ratio = (variance < 1e-6).sum() / seq.shape[1]

        return static_ratio < 0.3  # Moins de 30% de features statiques

    def __getitem__(self, idx):
        try:
            # Load the numpy array
            seq_np = np.load(self.samples[idx], mmap_mode='r', allow_pickle=False)

            # Ensure the data is float32 and handle potential non-finite values after loading
            seq_np = seq_np.astype(np.float32)
            if np.isnan(seq_np).any() or np.isinf(seq_np).any():
                 print(f"Warning: NaN or Inf found in {self.samples[idx]} after loading. Skipping.")
                 return None # Skip this sample

            seq = torch.from_numpy(seq_np)

            exercise_name = self.exercise_names[idx]
            # We no longer need expected_features per sample for padding in _standardize_sequence_enhanced

            # Validation physique if enabled
            if self.validate_physics and not self._validate_sample_quality(seq.numpy(), exercise_name):
                print(f"Warning: Sample {self.samples[idx]} failed quality validation. Skipping.")
                return None


            # Adaptive standardization using the global max_feature_dim
            seq = self._standardize_sequence_enhanced(seq, self.max_feature_dim)


            if self.augment:
                seq = self.augment_sequence(seq)

            return (
                seq.to(device),
                torch.tensor(self.labels[idx], dtype=torch.long).to(device),
                torch.tensor(self.groups[idx], dtype=torch.long).to(device)
            )

        except Exception as e:
            print(f"❌ Error loading or processing {self.samples[idx]}: {str(e)}")
            return None

    def _standardize_sequence_enhanced(self, seq, target_feature_dim):
        """Adaptive standardization to a target feature dimension."""
        # Feature adjustment
        if seq.shape[1] < target_feature_dim:
            padding = target_feature_dim - seq.shape[1]
            seq = torch.nn.functional.pad(seq, (0, padding), mode='constant', value=0)
        elif seq.shape[1] > target_feature_dim:
            seq = seq[:, :target_feature_dim]

        # Temporal adjustment
        if seq.shape[0] > self.max_seq_len:
            seq = seq[:self.max_seq_len]
        elif seq.shape[0] < self.max_seq_len:
            padding = self.max_seq_len - seq.shape[0]
            seq = torch.nn.functional.pad(seq, (0, 0, 0, padding), mode='constant', value=0)

        return seq

    def augment_sequence(self, seq):
      """Enhanced data augmentation"""
      # Controlled Jitter
      if random.random() < 0.5 and seq.numel() > 0: # Added check for empty tensor
          noise_level = 0.02 * torch.std(seq)
          seq += torch.randn_like(seq) * noise_level

      # Adaptive Scaling
      if random.random() < 0.5 and seq.numel() > 0: # Added check for empty tensor
          scale = random.uniform(0.95, 1.05)
          seq *= scale

      # Selective Masking with PyTorch
      if random.random() < 0.3 and seq.numel() > 0: # Added check for empty tensor
          mask = (torch.rand_like(seq) > 0.15).float()
          mean_val = torch.mean(seq) if seq.numel() > 0 else 0.0 # Handle empty tensor mean
          seq = seq * mask + (1 - mask) * mean_val

      # Temporal Permutation with PyTorch
      if random.random() < 0.2 and seq.size(0) > 5: # Ensure enough frames for splitting
          seq_len = seq.size(0)
          try:
              split_indices = sorted(random.sample(range(1, seq_len), random.randint(1, min(3, seq_len - 1))))
              split_sizes = [split_indices[0]]
              for i in range(len(split_indices) - 1):
                  split_sizes.append(split_indices[i+1] - split_indices[i])
              split_sizes.append(seq_len - split_indices[-1])
              segments = torch.split(seq, split_sizes)

              indices = torch.randperm(len(segments))
              seq = torch.cat([segments[i] for i in indices], dim=0)[:seq_len] # Ensure length is preserved
          except ValueError as e:
               print(f"Warning: Temporal permutation failed for a sample. {e}") # Log error

      return seq
import os
import numpy as np
from pathlib import Path
import shutil
from tqdm import tqdm

def validate_npy_file_flexible(file_path, max_seq_len=50, min_features=50,
                              max_features=300, tolerance_level="medium"):
    """
    Validation flexible avec différents niveaux de tolérance[2].

    Args:
        file_path (str): Chemin vers le fichier .npy
        tolerance_level (str): "strict", "medium", "relaxed"

    Returns:
        tuple: (bool, str, dict) - (Valide, Raison, Métriques)
    """
    try:
        data = np.load(file_path, allow_pickle=False, mmap_mode='r')

        if data is None or data.size == 0:
            return False, "Fichier vide", {}

        if len(data.shape) != 2:
            return False, f"Forme incorrecte: {data.shape}", {}

        sequence_length, num_features = data.shape

        # Critères adaptatifs selon le niveau de tolérance[4]
        if tolerance_level == "strict":
            seq_min, seq_max = 10, max_seq_len
            feat_min, feat_max = min_features, max_features
            static_threshold = 0.2
            variance_threshold = 1e-6
        elif tolerance_level == "medium":
            seq_min, seq_max = 5, max_seq_len + 10
            feat_min, feat_max = min_features - 20, max_features + 50
            static_threshold = 0.4
            variance_threshold = 1e-7
        else:  # relaxed
            seq_min, seq_max = 3, max_seq_len + 20
            feat_min, feat_max = min_features - 40, max_features + 100
            static_threshold = 0.6
            variance_threshold = 1e-8

        # Validation adaptative de la longueur
        if sequence_length < seq_min or sequence_length > seq_max:
            if tolerance_level == "relaxed" and sequence_length >= 3:
                pass  # Accepter même les très courtes séquences
            else:
                return False, f"Séquence: {sequence_length} hors limites [{seq_min}, {seq_max}]", {}

        # Validation adaptative des features
        if num_features < feat_min or num_features > feat_max:
            if tolerance_level == "relaxed":
                pass  # Accepter une plus large gamme
            else:
                return False, f"Features: {num_features} hors limites [{feat_min}, {feat_max}]", {}

        data_float = data.astype(np.float32)

        # Gestion flexible des valeurs NaN/Inf[5]
        nan_count = np.isnan(data_float).sum()
        inf_count = np.isinf(data_float).sum()
        total_values = data_float.size

        nan_ratio = nan_count / total_values
        inf_ratio = inf_count / total_values

        if tolerance_level == "strict":
            if nan_ratio > 0 or inf_ratio > 0:
                return False, "Valeurs NaN/Inf détectées", {"nan_ratio": nan_ratio, "inf_ratio": inf_ratio}
        elif tolerance_level == "medium":
            if nan_ratio > 0.05 or inf_ratio > 0.01:
                return False, f"Trop de NaN ({nan_ratio:.1%}) ou Inf ({inf_ratio:.1%})", {}
        else:  # relaxed
            if nan_ratio > 0.15 or inf_ratio > 0.05:
                return False, f"Trop de NaN ({nan_ratio:.1%}) ou Inf ({inf_ratio:.1%})", {}

        # Validation de variance adaptative[8]
        if not np.all(data_float == 0):
            feature_variances = np.var(data_float, axis=0)
            static_features_ratio = (feature_variances < variance_threshold).sum() / num_features

            if static_features_ratio > static_threshold:
                if tolerance_level != "relaxed":
                    return False, f"Features statiques: {static_features_ratio:.1%} > {static_threshold:.1%}", {}

        metrics = {
            "sequence_length": sequence_length,
            "num_features": num_features,
            "nan_ratio": nan_ratio,
            "inf_ratio": inf_ratio,
            "variance": np.var(data_float) if data_float.size > 0 else 0
        }

        return True, "Valide", metrics

    except Exception as e:
        return False, f"Erreur: {str(e)}", {}

def clean_invalid_npy_flexible(data_dir, backup_dir=None, tolerance_level="medium",
                              dry_run=False, verbose=True):
    """
    Pipeline de nettoyage avec validation flexible[4][7].
    """
    data_path = Path(data_dir)
    stats = {
        'total_files': 0,
        'invalid_files': 0,
        'valid_files': 0,
        'tolerance_level': tolerance_level,
        'processed_files': [],
        'invalid_reasons': {}
    }

    if backup_dir:
        backup_path = Path(backup_dir)
        backup_path.mkdir(parents=True, exist_ok=True)

    if verbose:
        print(f"🔍 NETTOYAGE FLEXIBLE - Niveau de tolérance: {tolerance_level.upper()}")
        print("=" * 60)

    npy_files = list(data_path.rglob("*.npy"))

    for npy_file in tqdm(npy_files, desc="Validation flexible", disable=not verbose):
        stats['total_files'] += 1

        is_valid, reason, metrics = validate_npy_file_flexible(
            npy_file,
            tolerance_level=tolerance_level
        )

        if not is_valid:
            stats['invalid_files'] += 1

            if reason not in stats['invalid_reasons']:
                stats['invalid_reasons'][reason] = 0
            stats['invalid_reasons'][reason] += 1

            if not dry_run and backup_dir:
                relative_path = npy_file.relative_to(data_path)
                backup_file_path = backup_path / relative_path
                backup_file_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.move(str(npy_file), str(backup_file_path))

            if verbose and stats['invalid_files'] <= 10:
                print(f"📦 {'[SIM]' if dry_run else 'Déplacé'}: {npy_file.name} - {reason}")
        else:
            stats['valid_files'] += 1
            stats['processed_files'].append(str(npy_file))

    if verbose:
        print("-" * 60)
        print(f"📊 Tolérance {tolerance_level}: {stats['valid_files']}/{stats['total_files']} fichiers valides")
        success_rate = (stats['valid_files'] / stats['total_files']) * 100 if stats['total_files'] > 0 else 0
        print(f"📈 Taux de réussite: {success_rate:.1f}%")

        if stats['invalid_reasons']:
            print("\n🔍 RAISONS D'INVALIDITÉ:")
            for reason, count in sorted(stats['invalid_reasons'].items(), key=lambda x: x[1], reverse=True):
                print(f"   • {reason}: {count} fichier(s)")

    return stats

def run_cleaning_pipeline(data_dir='/content/drive/MyDrive/preprocessed_videos_enhanced',
                         backup_dir='/content/drive/MyDrive/invalid_data_backup',
                         tolerance_level="medium"):
    """
    Fonction principale pour exécuter uniquement le pipeline de nettoyage[4][5].
    """
    print("🚀 LANCEMENT DU PIPELINE DE NETTOYAGE")
    print("=" * 80)

    # Étape 1: Simulation pour identifier les problèmes
    print("📋 ÉTAPE 1: SIMULATION")
    stats_simulation = clean_invalid_npy_flexible(
        data_dir=data_dir,
        backup_dir=backup_dir,
        tolerance_level=tolerance_level,
        dry_run=True,
        verbose=True
    )

    if not stats_simulation:
        print("❌ Erreur lors de la simulation.")
        return

    # Étape 2: Nettoyage réel si nécessaire
    if stats_simulation['invalid_files'] > 0:
        print(f"\n⚠️ {stats_simulation['invalid_files']} fichiers invalides détectés!")
        print("\n🔧 ÉTAPE 2: NETTOYAGE RÉEL")

        stats_real = clean_invalid_npy_flexible(
            data_dir=data_dir,
            backup_dir=backup_dir,
            tolerance_level=tolerance_level,
            dry_run=False,
            verbose=True
        )

        if stats_real:
            print(f"\n✅ Nettoyage terminé!")
            print(f"🧹 Dataset nettoyé: {stats_real['valid_files']} fichiers valides restants")

    else:
        print("\n✨ Aucun fichier invalide détecté! Votre dataset est déjà propre.")

    return stats_simulation

# Exécution directe du nettoyage
if __name__ == "__main__":
    # Configuration par défaut
    DATA_DIR = '/content/drive/MyDrive/preprocessed_videos_enhanced'
    BACKUP_DIR = '/content/drive/MyDrive/invalid_data_backup'

    # Lancement avec tolérance moyenne
    stats = run_cleaning_pipeline(
        data_dir=DATA_DIR,
        backup_dir=BACKUP_DIR,
        tolerance_level="medium"
    )


Nombre de groupes : 7
🚀 LANCEMENT DU PIPELINE DE NETTOYAGE
📋 ÉTAPE 1: SIMULATION
🔍 NETTOYAGE FLEXIBLE - Niveau de tolérance: MEDIUM


Validation flexible:   0%|          | 0/3213 [00:00<?, ?it/s]

📦 [SIM]: plank_3_2188552c_part1.npy - Features statiques: 40.3% > 40.0%


Validation flexible:   1%|          | 32/3213 [00:01<01:16, 41.40it/s]

📦 [SIM]: plank_6_part2_fe8f34d3_part3.npy - Features statiques: 40.3% > 40.0%
📦 [SIM]: plank_6_part3_ffb5fdb3_part1.npy - Features statiques: 57.1% > 40.0%
📦 [SIM]: plank_6_part3_ffb5fdb3_part2.npy - Features statiques: 45.5% > 40.0%
📦 [SIM]: plank_6_part3_ffb5fdb3_part3.npy - Features statiques: 61.0% > 40.0%
📦 [SIM]: plank_6_part4_fab7a22d_part1.npy - Features statiques: 62.3% > 40.0%
📦 [SIM]: plank_6_part4_fab7a22d_part3.npy - Features statiques: 42.9% > 40.0%


...

📦 Déplacé: plank_6_part3_ffb5fdb3_part3.npy - Features statiques: 61.0% > 40.0%
📦 Déplacé: plank_6_part4_fab7a22d_part1.npy - Features statiques: 62.3% > 40.0%
📦 Déplacé: plank_6_part4_fab7a22d_part3.npy - Features statiques: 42.9% > 40.0%
📦 Déplacé: plank_6_part9_5534d62f_part1.npy - Features statiques: 41.6% > 40.0%
📦 Déplacé: plank_6_part9_5534d62f_part2.npy - Features statiques: 45.5% > 40.0%


Validation flexible:   2%|▏         | 58/3213 [00:00<00:31, 99.37it/s]

📦 Déplacé: plank_6_part13_53ce6e05_part1.npy - Features statiques: 40.3% > 40.0%


Validation flexible: 100%|██████████| 3213/3213 [00:30<00:00, 103.68it/s]

------------------------------------------------------------
📊 Tolérance medium: 3136/3213 fichiers valides
📈 Taux de réussite: 97.6%

🔍 RAISONS D'INVALIDITÉ:
   • Features statiques: 40.3% > 40.0%: 10 fichier(s)
   • Features statiques: 42.9% > 40.0%: 7 fichier(s)
   • Features statiques: 61.0% > 40.0%: 6 fichier(s)
   • Features statiques: 41.6% > 40.0%: 6 fichier(s)
   • Features statiques: 54.5% > 40.0%: 6 fichier(s)
   • Features statiques: 44.2% > 40.0%: 6 fichier(s)
   • Features statiques: 50.6% > 40.0%: 6 fichier(s)
   • Features statiques: 53.2% > 40.0%: 6 fichier(s)
   • Features statiques: 46.8% > 40.0%: 5 fichier(s)
   • Features statiques: 45.5% > 40.0%: 4 fichier(s)
   • Features statiques: 62.3% > 40.0%: 3 fichier(s)
   • Features statiques: 76.6% > 40.0%: 3 fichier(s)
   • Features statiques: 48.1% > 40.0%: 3 fichier(s)
   • Features statiques: 57.1% > 40.0%: 2 fichier(s)
   • Features statiques: 49.4% > 40.0%: 2 fichier(s)
   • Features statiques: 51.9% > 40.0%: 1 fic

In [4]:
import os
import numpy as np
from pathlib import Path
import shutil
from tqdm import tqdm
import json

def restore_files_from_backup(backup_dir, original_dir, dry_run=False, verbose=True):
    """
    Restaure tous les fichiers .npy depuis le répertoire de sauvegarde vers leur emplacement d'origine.

    Args:
        backup_dir (str): Répertoire de sauvegarde contenant les fichiers déplacés
        original_dir (str): Répertoire d'origine où restaurer les fichiers
        dry_run (bool): Si True, effectue seulement une simulation
        verbose (bool): Afficher les détails du processus

    Returns:
        dict: Statistiques de restauration
    """
    backup_path = Path(backup_dir)
    original_path = Path(original_dir)

    # Vérification de l'existence des répertoires
    if not backup_path.exists():
        print(f"❌ Erreur: Le répertoire de sauvegarde {backup_dir} n'existe pas.")
        return None

    if not original_path.exists():
        print(f"📁 Création du répertoire d'origine: {original_dir}")
        if not dry_run:
            original_path.mkdir(parents=True, exist_ok=True)

    stats = {
        'total_files': 0,
        'restored_files': 0,
        'errors': [],
        'restored_paths': [],
        'skipped_files': []
    }

    if verbose:
        print("=" * 80)
        print("🔄 RESTAURATION DES FICHIERS DEPUIS LA SAUVEGARDE")
        print("=" * 80)
        print(f"📁 Répertoire de sauvegarde: {backup_dir}")
        print(f"📁 Répertoire de destination: {original_dir}")
        print(f"📋 Mode simulation: {dry_run}")
        print("-" * 80)

    # Parcourir tous les fichiers .npy dans le backup récursivement
    npy_files = list(backup_path.rglob("*.npy"))

    if not npy_files:
        print(f"⚠️ Aucun fichier .npy trouvé dans {backup_dir}")
        return stats

    for backup_file in tqdm(npy_files, desc="🔄 Restauration", disable=not verbose):
        stats['total_files'] += 1

        # Calculer le chemin relatif depuis le backup
        relative_path = backup_file.relative_to(backup_path)
        target_file = original_path / relative_path

        # Créer le répertoire parent si nécessaire
        target_file.parent.mkdir(parents=True, exist_ok=True)

        if target_file.exists():
            stats['skipped_files'].append(str(target_file))
            if verbose:
                print(f"⚠️ Fichier existe déjà: {target_file.name}")
            continue

        if not dry_run:
            try:
                # Déplacer le fichier depuis le backup vers l'original
                shutil.move(str(backup_file), str(target_file))
                stats['restored_files'] += 1
                stats['restored_paths'].append(str(target_file))

                if verbose and stats['restored_files'] <= 10:
                    print(f"✅ Restauré: {relative_path}")

            except Exception as e:
                error_msg = f"Erreur lors de la restauration de {backup_file}: {e}"
                stats['errors'].append(error_msg)
                if verbose:
                    print(f"❌ {error_msg}")
        else:
            stats['restored_files'] += 1
            if verbose and stats['restored_files'] <= 10:
                print(f"🔍 [SIMULATION] Restauration: {relative_path}")

    # Affichage du résumé
    if verbose:
        print("-" * 80)
        print("📊 RÉSUMÉ DE LA RESTAURATION:")
        print(f"   📄 Total des fichiers trouvés: {stats['total_files']}")
        print(f"   ✅ Fichiers restaurés: {stats['restored_files']}")
        print(f"   ⚠️ Fichiers ignorés (déjà existants): {len(stats['skipped_files'])}")
        print(f"   ❌ Erreurs: {len(stats['errors'])}")

        if stats['errors']:
            print("\n⚠️ ERREURS RENCONTRÉES:")
            for error in stats['errors'][:5]:
                print(f"   • {error}")

        print("=" * 80)

        if not dry_run and stats['restored_files'] > 0:
            print(f"✅ Restauration terminée! {stats['restored_files']} fichiers restaurés.")
        elif dry_run:
            print(f"🔍 Simulation terminée. {stats['restored_files']} fichiers seraient restaurés.")

    return stats

def cleanup_empty_backup_directories(backup_dir, dry_run=False, verbose=True):
    """
    Supprime les répertoires vides dans le backup après restauration[9].

    Args:
        backup_dir (str): Répertoire de sauvegarde à nettoyer
        dry_run (bool): Mode simulation
        verbose (bool): Affichage détaillé
    """
    backup_path = Path(backup_dir)
    removed_dirs = []

    if verbose:
        print("\n🧹 NETTOYAGE DES RÉPERTOIRES VIDES")
        print("-" * 40)

    # Parcourir les répertoires de bas en haut pour supprimer les vides
    for dirpath in sorted(backup_path.rglob("*"), key=lambda p: len(p.parts), reverse=True):
        if dirpath.is_dir() and not any(dirpath.iterdir()):
            if not dry_run:
                try:
                    dirpath.rmdir()
                    removed_dirs.append(str(dirpath))
                    if verbose:
                        print(f"🗑️ Répertoire vide supprimé: {dirpath.name}")
                except Exception as e:
                    if verbose:
                        print(f"❌ Erreur suppression {dirpath}: {e}")
            else:
                removed_dirs.append(str(dirpath))
                if verbose:
                    print(f"🔍 [SIMULATION] Suppression: {dirpath.name}")

    if verbose:
        print(f"📊 {len(removed_dirs)} répertoires vides {'supprimés' if not dry_run else 'à supprimer'}")

    return removed_dirs

def run_restoration_pipeline():
    """
    Pipeline principal de restauration des fichiers[13].
    """
    # Configuration - Ajustez ces chemins selon votre configuration
    BACKUP_DIR = '/content/drive/MyDrive/invalid_data_backup'
    ORIGINAL_DIR = '/content/drive/MyDrive/preprocessed_videos_enhanced'

    print("🚀 LANCEMENT DU PIPELINE DE RESTAURATION")
    print("=" * 80)

    # Étape 1: Simulation de la restauration
    print("📋 ÉTAPE 1: SIMULATION DE LA RESTAURATION")
    stats_simulation = restore_files_from_backup(
        backup_dir=BACKUP_DIR,
        original_dir=ORIGINAL_DIR,
        dry_run=True,
        verbose=True
    )

    if not stats_simulation:
        print("❌ Erreur lors de la simulation.")
        return

    # Étape 2: Restauration réelle si des fichiers sont trouvés
    if stats_simulation['total_files'] > 0:
        print(f"\n🔄 {stats_simulation['total_files']} fichiers trouvés dans la sauvegarde!")
        print("\n🔧 ÉTAPE 2: RESTAURATION RÉELLE")

        # Restauration automatique
        stats_real = restore_files_from_backup(
            backup_dir=BACKUP_DIR,
            original_dir=ORIGINAL_DIR,
            dry_run=False,
            verbose=True
        )

        if stats_real and stats_real['restored_files'] > 0:
            print(f"\n✅ Restauration terminée!")
            print(f"🔄 {stats_real['restored_files']} fichiers restaurés dans {ORIGINAL_DIR}")

            # Étape 3: Nettoyage des répertoires vides dans le backup
            print("\n🧹 ÉTAPE 3: NETTOYAGE DU BACKUP")
            cleanup_empty_backup_directories(
                backup_dir=BACKUP_DIR,
                dry_run=False,
                verbose=True
            )

    else:
        print("\n⚠️ Aucun fichier trouvé dans le répertoire de sauvegarde.")
        print("Vérifiez le chemin du répertoire de sauvegarde.")

    return stats_simulation

# Exécution du pipeline
if __name__ == "__main__":
    run_restoration_pipeline()


🚀 LANCEMENT DU PIPELINE DE RESTAURATION
📋 ÉTAPE 1: SIMULATION DE LA RESTAURATION
🔄 RESTAURATION DES FICHIERS DEPUIS LA SAUVEGARDE
📁 Répertoire de sauvegarde: /content/drive/MyDrive/invalid_data_backup
📁 Répertoire de destination: /content/drive/MyDrive/preprocessed_videos_enhanced
📋 Mode simulation: True
--------------------------------------------------------------------------------


🔄 Restauration: 100%|██████████| 77/77 [00:00<00:00, 2107.77it/s]


🔍 [SIMULATION] Restauration: plank/plank_3_2188552c_part1.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part2_fe8f34d3_part3.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part3_ffb5fdb3_part1.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part3_ffb5fdb3_part2.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part3_ffb5fdb3_part3.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part4_fab7a22d_part1.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part4_fab7a22d_part3.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part9_5534d62f_part1.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part9_5534d62f_part2.npy
🔍 [SIMULATION] Restauration: plank/plank_6_part13_53ce6e05_part1.npy
--------------------------------------------------------------------------------
📊 RÉSUMÉ DE LA RESTAURATION:
   📄 Total des fichiers trouvés: 77
   ✅ Fichiers restaurés: 77
   ⚠️ Fichiers ignorés (déjà existants): 0
   ❌ Erreurs: 0
🔍 Simulation terminée. 77 fichiers seraient restaurés.

🔄 77 fichiers trouvés dans la sauveg

🔄 Restauration:  35%|███▌      | 27/77 [00:00<00:00, 262.42it/s]

✅ Restauré: plank/plank_3_2188552c_part1.npy
✅ Restauré: plank/plank_6_part2_fe8f34d3_part3.npy
✅ Restauré: plank/plank_6_part3_ffb5fdb3_part1.npy
✅ Restauré: plank/plank_6_part3_ffb5fdb3_part2.npy
✅ Restauré: plank/plank_6_part3_ffb5fdb3_part3.npy
✅ Restauré: plank/plank_6_part4_fab7a22d_part1.npy
✅ Restauré: plank/plank_6_part4_fab7a22d_part3.npy
✅ Restauré: plank/plank_6_part9_5534d62f_part1.npy
✅ Restauré: plank/plank_6_part9_5534d62f_part2.npy
✅ Restauré: plank/plank_6_part13_53ce6e05_part1.npy


🔄 Restauration: 100%|██████████| 77/77 [00:00<00:00, 267.86it/s]

--------------------------------------------------------------------------------
📊 RÉSUMÉ DE LA RESTAURATION:
   📄 Total des fichiers trouvés: 77
   ✅ Fichiers restaurés: 77
   ⚠️ Fichiers ignorés (déjà existants): 0
   ❌ Erreurs: 0
✅ Restauration terminée! 77 fichiers restaurés.

✅ Restauration terminée!
🔄 77 fichiers restaurés dans /content/drive/MyDrive/preprocessed_videos_enhanced

🧹 ÉTAPE 3: NETTOYAGE DU BACKUP

🧹 NETTOYAGE DES RÉPERTOIRES VIDES
----------------------------------------
🗑️ Répertoire vide supprimé: plank
📊 1 répertoires vides supprimés


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os
from tqdm import tqdm
import json # To save mappings
from pathlib import Path # For path manipulation
import warnings
warnings.filterwarnings('ignore')


# Assuming EnhancedExerciseHierarchicalDataset, exercises_points, group_mapping, and device are defined in a previous cell

class PoseClassificationModel(nn.Module):
    """
    Modèle de classification de pose hiérarchique simple utilisant des RNN/LSTM
    pour les séquences, suivi de couches linéaires pour les tâches
    de classification de groupe et d'exercice.
    """
    def __init__(self, input_dim, hidden_dim, num_groups, num_exercises):
        super(PoseClassificationModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_groups = num_groups
        self.num_exercises = num_exercises

        # Couche RNN/LSTM pour traiter la séquence de landmarks
        # Utilise LSTM pour capturer les dépendances temporelles
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        # Couche linéaire pour la classification du groupe (sortie de l'état caché final du LSTM)
        self.group_classifier = nn.Linear(hidden_dim, num_groups)

        # Couche linéaire pour la classification de l'exercice (sortie de l'état caché final du LSTM)
        self.exercise_classifier = nn.Linear(hidden_dim, num_exercises)

    def forward(self, sequence):
        # sequence: Tensor de forme (batch_size, sequence_length, input_dim)

        # Passer la séquence à travers le LSTM
        out, (hn, cn) = self.lstm(sequence)

        # Nous utilisons l'état caché du DERNIER pas temporel pour la classification
        # hn: tenseur de forme (num_layers * num_directions, batch_size, hidden_dim)
        # Si batch_first=True et num_layers=1, hn est de forme (1, batch_size, hidden_dim)
        # Nous prenons l'état caché de la dernière couche
        final_hidden_state = hn[-1, :, :]

        # Passer l'état caché final à travers les classificateurs
        group_output = self.group_classifier(final_hidden_state)
        exercise_output = self.exercise_classifier(final_hidden_state)

        return group_output, exercise_output

# --- Préparation des données et de l'entraînement ---

# Configuration du Dataset (ajuster les chemins)
dataset_config = {
    'data_dir': '/content/drive/MyDrive/preprocessed_videos_enhanced', # Le répertoire où les fichiers .npy sont sauvegardés
    'max_seq_len': 50, # Adjusted based on previous analysis
    'augment': True, # Utiliser l'augmentation de données
    'validate_physics': True # Utiliser la validation de qualité physique au chargement
}

# Instanciation du Dataset amélioré
full_dataset = EnhancedExerciseHierarchicalDataset(
    data_dir=dataset_config['data_dir'],
    exercices_points=exercises_points, # Assuming exercises_points is available
    grouped_sports=group_mapping,     # Assuming group_mapping is available
    max_seq_len=dataset_config['max_seq_len'],
    augment=dataset_config['augment'],
    validate_physics=dataset_config['validate_physics']
)

# Gérer le cas où aucun échantillon valide n'est chargé
if len(full_dataset) == 0:
    print("Erreur: Aucun échantillon valide n'a été chargé. Vérifiez le chemin des données et la configuration.")
else:
    print(f"Dataset chargé avec {len(full_dataset)} échantillons valides.")

    # Récupérer la dimension calculée par le dataset
    input_dim = full_dataset.max_feature_dim
    num_groups = len(full_dataset.group_to_idx)
    num_exercises = len(full_dataset.exercise_to_idx)

    print(f"Dimension des features (calculée par le dataset): {input_dim}")
    print(f"Nombre de groupes détectés: {num_groups}")
    print(f"Nombre d'exercices détectés: {num_exercises}")


    # Définir les hyperparamètres du modèle
    hidden_dim = 128 # Taille de l'état caché du LSTM


    # Instanciation du modèle
    model = PoseClassificationModel(input_dim, hidden_dim, num_groups, num_exercises).to(device)
    print("Modèle instancié:")
    print(model)
    print(f"Nombre de paramètres entraînabels: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

    # Critères de perte pour les deux tâches
    group_criterion = nn.CrossEntropyLoss()
    exercise_criterion = nn.CrossEntropyLoss()

    # Optimiseur (Adam est un choix courant)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Learning Rate Scheduler (StepLR for adaptive learning rate with step=200)
    # Adjust gamma (multiplicative factor) and step_size as needed
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)


    # Fractionnement du dataset (70% entraînement, 20% validation, 10% test)
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size # Ensure all samples are used

    if train_size == 0 or val_size == 0:
         print("Erreur: Les tailles d'entraînement ou de validation sont de 0. Le dataset est peut-être trop petit.")
    else:
        # Use generator for reproducibility if needed
        # generator = torch.Generator().manual_seed(42)
        train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size]) #, generator=generator)

        print(f"Dataset split: Train ({len(train_dataset)}), Validation ({len(val_dataset)}), Test ({len(test_dataset)})")


        # Création des DataLoaders (pour gérer le batching et le brassage)
        def collate_fn(batch):
            # Filter out None samples (due to loading or validation errors)
            batch = list(filter(lambda x: x is not None, batch))
            if not batch: # Return None if the batch is empty after filtering
                return None
            sequences, labels, groups = zip(*batch)
            # Sequences are already padded to max_seq_len in __getitem__
            padded_sequences = torch.stack(sequences)
            return padded_sequences, torch.stack(labels), torch.stack(groups)


        batch_size = 32 # Taille du batch

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2) # Added num_workers
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=2) # Added num_workers
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=2) # Added num_workers


        # --- Boucle d'entraînement avec checkpointing et métriques ---

        num_epochs = 50 # Increased epochs for potentially better training

        print("\nDébut de l'entraînement...")

        best_val_accuracy = 0.0
        best_model_path = '/content/drive/MyDrive/best_pose_classification_model1.pth'
        mapping_path = '/content/drive/MyDrive/pose_classification_mappings1.json'

        # Save mappings
        mappings_to_save = {
            'exercise_points': exercises_points,
            'group_mapping': group_mapping,
            'exercise_to_idx': full_dataset.exercise_to_idx,
            'idx_to_exercise': full_dataset.idx_to_exercise,
            'group_to_idx': full_dataset.group_to_idx,
            'idx_to_group': full_dataset.idx_to_group
        }
        with open(mapping_path, 'w') as f:
             # Convert Path objects to strings for JSON serialization
             json.dump(mappings_to_save, f, indent=4)

        print(f"Mappings saved to {mapping_path}")

        for epoch in range(num_epochs):
            model.train() # Mettre le modèle en mode entraînement
            running_loss = 0.0
            running_group_loss = 0.0
            running_exercise_loss = 0.0
            total_samples = 0

            # Using tqdm for progress bar
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]") as pbar:
                for batch in pbar:
                    if batch is None: # Skip empty batches
                        continue
                    sequences, exercise_labels, group_labels = batch

                    # Remettre les gradients à zéro
                    optimizer.zero_grad()

                    # Passage avant (forward pass)
                    group_outputs, exercise_outputs = model(sequences)

                    # Calculer la perte pour les deux tâches
                    group_loss = group_criterion(group_outputs, group_labels)
                    exercise_loss = exercise_criterion(exercise_outputs, exercise_labels)

                    # Combiner les pertes
                    total_loss = group_loss + exercise_loss # Simple sum for now

                    # Rétropropagation (backward pass)
                    total_loss.backward()

                    # Mettre à jour les poids du modèle
                    optimizer.step()

                    # Update learning rate
                    scheduler.step()


                    running_loss += total_loss.item() * sequences.size(0)
                    running_group_loss += group_loss.item() * sequences.size(0)
                    running_exercise_loss += exercise_loss.item() * sequences.size(0)
                    total_samples += sequences.size(0)

                    # Update progress bar description with current loss
                    pbar.set_postfix({
                        'loss': running_loss / total_samples if total_samples > 0 else 0.0,
                        'lr': scheduler.get_last_lr()[0]
                    })


            epoch_loss = running_loss / total_samples if total_samples > 0 else 0
            epoch_group_loss = running_group_loss / total_samples if total_samples > 0 else 0
            epoch_exercise_loss = running_exercise_loss / total_samples if total_samples > 0 else 0


            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f} (Group: {epoch_group_loss:.4f}, Exercise: {epoch_exercise_loss:.4f})")

            # --- Validation ---
            model.eval() # Mettre le modèle en mode évaluation
            val_running_loss = 0.0
            val_running_group_loss = 0.0
            val_running_exercise_loss = 0.0
            correct_groups = 0
            correct_exercises = 0
            total_val_samples = 0

            with torch.no_grad(): # Désactiver le calcul des gradients pendant la validation
                with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]") as pbar:
                    for batch in pbar:
                        if batch is None: # Skip empty batches
                            continue
                        sequences, exercise_labels, group_labels = batch

                        group_outputs, exercise_outputs = model(sequences)

                        group_loss = group_criterion(group_outputs, group_labels)
                        exercise_loss = exercise_criterion(exercise_outputs, exercise_labels)

                        total_loss = group_loss + exercise_loss

                        val_running_loss += total_loss.item() * sequences.size(0)
                        val_running_group_loss += group_loss.item() * sequences.size(0)
                        val_running_exercise_loss += exercise_loss.item() * sequences.size(0)
                        total_val_samples += sequences.size(0)

                        # Calcul de l'accuracy
                        _, predicted_groups = torch.max(group_outputs, 1)
                        correct_groups += (predicted_groups == group_labels).sum().item()

                        _, predicted_exercises = torch.max(exercise_outputs, 1)
                        correct_exercises += (predicted_exercises == exercise_labels).sum().item()

                        # Update progress bar description
                        pbar.set_postfix({
                            'loss': val_running_loss / total_val_samples if total_val_samples > 0 else 0.0,
                            'group_acc': correct_groups / total_val_samples * 100 if total_val_samples > 0 else 0.0,
                            'exercise_acc': correct_exercises / total_val_samples * 100 if total_val_samples > 0 else 0.0
                        })


            val_epoch_loss = val_running_loss / total_val_samples if total_val_samples > 0 else 0
            val_epoch_group_loss = val_running_group_loss / total_val_samples if total_val_samples > 0 else 0
            val_epoch_exercise_loss = val_running_exercise_loss / total_val_samples if total_val_samples > 0 else 0
            val_group_accuracy = correct_groups / total_val_samples if total_val_samples > 0 else 0
            val_exercise_accuracy = correct_exercises / total_val_samples if total_val_samples > 0 else 0


            print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_epoch_loss:.4f} (Group: {val_epoch_group_loss:.4f}, Exercise: {val_epoch_exercise_loss:.4f})")
            print(f"                 Val Accuracy: Group: {val_group_accuracy:.2%}, Exercise: {val_exercise_accuracy:.2%}") # Report as percentage


            # --- Checkpointing the best model ---
            # Using exercise accuracy as the metric to monitor
            if val_exercise_accuracy > best_val_accuracy:
                best_val_accuracy = val_exercise_accuracy
                torch.save(model.state_dict(), best_model_path)
                print(f"➡️ Saved best model checkpoint with Exercise Accuracy: {best_val_accuracy:.2%}")

        print("\nEntraînement terminé.")
        print(f"Meilleure précision d'exercice de validation : {best_val_accuracy:.2%}")


        # --- Evaluation on the Test Set ---
        if len(test_dataset) > 0:
            print("\nDébut de l'évaluation sur l'ensemble de test...")
            model.eval() # Mettre le modèle en mode évaluation
            test_running_loss = 0.0
            test_running_group_loss = 0.0
            test_running_exercise_loss = 0.0
            test_correct_groups = 0
            test_correct_exercises = 0
            total_test_samples = 0

            with torch.no_grad(): # Désactiver le calcul des gradients
                 with tqdm(test_loader, desc="[Test Evaluation]") as pbar:
                    for batch in pbar:
                        if batch is None: # Skip empty batches
                            continue
                        sequences, exercise_labels, group_labels = batch

                        group_outputs, exercise_outputs = model(sequences)

                        group_loss = group_criterion(group_outputs, group_labels)
                        exercise_loss = exercise_criterion(exercise_outputs, exercise_labels)

                        total_loss = group_loss + exercise_loss

                        test_running_loss += total_loss.item() * sequences.size(0)
                        test_running_group_loss += group_loss.item() * sequences.size(0)
                        test_running_exercise_loss += exercise_loss.item() * sequences.size(0)
                        total_test_samples += sequences.size(0)

                        # Calcul de l'accuracy
                        _, predicted_groups = torch.max(group_outputs, 1)
                        test_correct_groups += (predicted_groups == group_labels).sum().item()

                        _, predicted_exercises = torch.max(exercise_outputs, 1)
                        test_correct_exercises += (predicted_exercises == exercise_labels).sum().item()

                        # Update progress bar description
                        pbar.set_postfix({
                             'loss': test_running_loss / total_test_samples if total_test_samples > 0 else 0.0,
                             'group_acc': test_correct_groups / total_test_samples * 100 if total_test_samples > 0 else 0.0,
                             'exercise_acc': test_correct_exercises / total_test_samples * 100 if total_test_samples > 0 else 0.0
                         })


            test_epoch_loss = test_running_loss / total_test_samples if total_test_samples > 0 else 0
            test_epoch_group_loss = test_running_group_loss / total_test_samples if total_test_samples > 0 else 0
            test_epoch_exercise_loss = test_running_exercise_loss / total_test_samples if total_test_samples > 0 else 0
            test_group_accuracy = test_correct_groups / total_test_samples if total_test_samples > 0 else 0
            test_exercise_accuracy = test_correct_exercises / total_test_samples if total_test_samples > 0 else 0

            print(f"\nRésultats sur l'ensemble de test:")
            print(f"  Test Loss: {test_epoch_loss:.4f} (Group: {test_epoch_group_loss:.4f}, Exercise: {test_epoch_exercise_loss:.4f})")
            print(f"  Test Accuracy: Group: {test_group_accuracy:.2%}, Exercise: {test_exercise_accuracy:.2%}") # Report as percentage

        else:
            print("\nPas d'ensemble de test pour l'évaluation.")

        print("\nProcessus d'entraînement et d'évaluation terminé.")

Scanning directory for preprocessed data: /content/drive/MyDrive/preprocessed_videos_enhanced
Found 22 present exercises with data and group mapping.
exercise_to_idx (dynamic): {'barbell biceps curl': 0, 'bench press': 1, 'chest fly machine': 2, 'deadlift': 3, 'decline bench press': 4, 'hammer curl': 5, 'hip thrust': 6, 'incline bench press': 7, 'lat pulldown': 8, 'lateral raise': 9, 'leg extension': 10, 'leg raises': 11, 'plank': 12, 'pull up': 13, 'push-up': 14, 'romanian deadlift': 15, 'russian twist': 16, 'shoulder press': 17, 'squat': 18, 't bar row': 19, 'tricep dips': 20, 'tricep pushdown': 21}
exercise_to_group (dynamic): {'bench press': 'horizontal pushes', 'incline bench press': 'horizontal pushes', 'push-up': 'horizontal pushes', 'shoulder press': 'vertical pushes', 'tricep dips': 'vertical pushes', 'tricep pushdown': 'vertical pushes', 'lateral raise': 'vertical pushes', 't bar row': 'horizontal pulls', 'barbell biceps curl': 'horizontal pulls', 'hammer curl': 'horizontal p

Epoch 1/50 [Train]:   0%|          | 0/71 [00:00<?, ?it/s]

Epoch 1/50 [Train]:   1%|▏         | 1/71 [00:00<00:42,  1.63it/s, loss=5.05, lr=0.001]

Epoch 1/50 [Train]:   7%|▋         | 5/71 [00:01<00:22,  3.00it/s, loss=5, lr=0.001]   

Epoch 1/50 [Train]:  10%|▉         | 7/71 [00:02<00:19,  3.35it/s, loss=4.99, lr=0.001]

...

Epoch 2/50 [Train]:  46%|████▋     | 33/71 [00:09<00:13,  2.84it/s, loss=2.12, lr=0.001]

Epoch 2/50 [Train]:  46%|████▋     | 33/71 [00:10<00:13,  2.84it/s, loss=2.11, lr=0.001]

Epoch 2/50 [Train]:  49%|████▉     | 35/71 [00:10<00:12,  2.81it/s, loss=2.11, lr=0.001]

Epoch 2/50 [Train]:  51%|█████     | 36/71 [00:11<00:10,  3.19it/s, loss=2.11, lr=0.001]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os
from tqdm import tqdm
import json
from pathlib import Path
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# ========================================================
# GESTIONNAIRE DE CHECKPOINTS AVEC REPRISE D'ENTRAÎNEMENT
# ========================================================

class EnhancedCheckpointManager:
    """
    Gestionnaire de checkpoints permettant la reprise exacte de l'entraînement.
    """

    def __init__(self, checkpoint_dir, model_name="model", save_top_k=3):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.model_name = model_name
        self.save_top_k = save_top_k
        self.best_checkpoints = []
        self.training_history = {
            'train_loss': [],
            'val_loss': [],
            'train_group_loss': [],
            'train_exercise_loss': [],
            'val_group_loss': [],
            'val_exercise_loss': [],
            'train_group_acc': [],
            'train_exercise_acc': [],
            'val_group_acc': [],
            'val_exercise_acc': [],
            'learning_rates': [],
            'epochs': [],
            'timestamps': []
        }
        self.best_score = 0.0

    def save_checkpoint(self, model, optimizer, scheduler, epoch,
                       train_metrics, val_metrics, is_best=False):
        """
        Sauvegarde un checkpoint complet pour reprise d'entraînement[6].
        """
        timestamp = datetime.now().isoformat()

        checkpoint_data = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'train_metrics': train_metrics,
            'val_metrics': val_metrics,
            'training_history': self.training_history,
            'best_score': self.best_score,
            'timestamp': timestamp,
            'model_config': {
                'input_dim': model.lstm.input_size,
                'hidden_dim': model.hidden_dim,
                'num_groups': model.num_groups,
                'num_exercises': model.num_exercises
            }
        }

        # Mettre à jour l'historique d'entraînement
        self.training_history['epochs'].append(epoch)
        self.training_history['timestamps'].append(timestamp)
        self.training_history['train_loss'].append(train_metrics['loss'])
        self.training_history['val_loss'].append(val_metrics['loss'])
        self.training_history['train_group_loss'].append(train_metrics['group_loss'])
        self.training_history['train_exercise_loss'].append(train_metrics['exercise_loss'])
        self.training_history['val_group_loss'].append(val_metrics['group_loss'])
        self.training_history['val_exercise_loss'].append(val_metrics['exercise_loss'])
        self.training_history['val_group_acc'].append(val_metrics['group_acc'])
        self.training_history['val_exercise_acc'].append(val_metrics['exercise_acc'])
        self.training_history['learning_rates'].append(optimizer.param_groups[0]['lr'])

        # Chemins des checkpoints
        current_checkpoint_path = self.checkpoint_dir / f"{self.model_name}_epoch_{epoch:03d}.pth"
        latest_checkpoint_path = self.checkpoint_dir / f"{self.model_name}_latest.pth"

        # Sauvegardes avec gestion d'erreurs[11]
        torch.save(checkpoint_data, current_checkpoint_path)
        torch.save(checkpoint_data, latest_checkpoint_path)

        if is_best:
            best_checkpoint_path = self.checkpoint_dir / f"{self.model_name}_best.pth"
            torch.save(checkpoint_data, best_checkpoint_path)

            val_exercise_acc = val_metrics['exercise_acc']
            self.best_score = val_exercise_acc

            checkpoint_info = {
                'epoch': epoch,
                'path': str(current_checkpoint_path),
                'val_exercise_acc': val_exercise_acc,
                'val_group_acc': val_metrics['group_acc'],
                'val_loss': val_metrics['loss'],
                'timestamp': timestamp
            }

            self.best_checkpoints.append(checkpoint_info)
            self.best_checkpoints.sort(key=lambda x: x['val_exercise_acc'], reverse=True)

            if len(self.best_checkpoints) > self.save_top_k:
                to_remove = self.best_checkpoints[self.save_top_k:]
                self.best_checkpoints = self.best_checkpoints[:self.save_top_k]

                for checkpoint_info in to_remove:
                    checkpoint_path = Path(checkpoint_info['path'])
                    if checkpoint_path.exists():
                        checkpoint_path.unlink()

        self._save_training_metadata()

        print(f"💾 Checkpoint sauvegardé: epoch_{epoch:03d}.pth")
        if is_best:
            print(f"🏆 Nouveau meilleur modèle! Exercise Accuracy: {val_metrics['exercise_acc']:.2%}")

    def _save_training_metadata(self):
        """Sauvegarde les métadonnées d'entraînement[11]."""
        history_path = self.checkpoint_dir / f"{self.model_name}_training_history.json"
        with open(history_path, 'w') as f:
            json.dump(self.training_history, f, indent=4)

        best_checkpoints_path = self.checkpoint_dir / f"{self.model_name}_best_checkpoints.json"
        with open(best_checkpoints_path, 'w') as f:
            json.dump(self.best_checkpoints, f, indent=4)

    def load_checkpoint(self, model, optimizer=None, scheduler=None, checkpoint_path=None, load_best=False):
        """
        Charge un checkpoint pour reprendre l'entraînement[6][13].
        """
        if checkpoint_path is None:
            if load_best:
                checkpoint_path = self.checkpoint_dir / f"{self.model_name}_best.pth"
            else:
                checkpoint_path = self.checkpoint_dir / f"{self.model_name}_latest.pth"

        if not Path(checkpoint_path).exists():
            print(f"⚠️ Aucun checkpoint trouvé: {checkpoint_path}")
            return 0

        checkpoint = torch.load(checkpoint_path, map_location='cpu')

        model.load_state_dict(checkpoint['model_state_dict'])

        if optimizer and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        if scheduler and 'scheduler_state_dict' in checkpoint and checkpoint['scheduler_state_dict']:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        self.training_history = checkpoint.get('training_history', self.training_history)
        self.best_score = checkpoint.get('best_score', 0.0)

        print(f"✅ Checkpoint chargé: {checkpoint_path}")
        print(f"📊 Reprise depuis l'époque: {checkpoint['epoch']}, Meilleur score: {self.best_score:.2%}")

        return checkpoint['epoch']

    def check_for_existing_checkpoint(self):
        """
        Vérifie s'il existe un checkpoint pour reprendre l'entraînement[6].
        """
        latest_path = self.checkpoint_dir / f"{self.model_name}_latest.pth"
        return latest_path.exists()

# ===============================================
# MODÈLE DE CLASSIFICATION DE POSE
# ===============================================

class PoseClassificationModel(nn.Module):
    """
    Modèle de classification de pose hiérarchique utilisant LSTM[15].
    """
    def __init__(self, input_dim, hidden_dim, num_groups, num_exercises):
        super(PoseClassificationModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_groups = num_groups
        self.num_exercises = num_exercises

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.group_classifier = nn.Linear(hidden_dim, num_groups)
        self.exercise_classifier = nn.Linear(hidden_dim, num_exercises)

    def forward(self, sequence):
        out, (hn, cn) = self.lstm(sequence)
        final_hidden_state = hn[-1, :, :]

        group_output = self.group_classifier(final_hidden_state)
        exercise_output = self.exercise_classifier(final_hidden_state)

        return group_output, exercise_output

# ========================================================
# FONCTION D'ENTRAÎNEMENT AVEC REPRISE AUTOMATIQUE
# ========================================================

def train_model_with_resume(model, train_loader, val_loader, num_epochs=50,
                           checkpoint_dir='/content/drive/MyDrive/checkpoints_enhanced',
                           device='cuda', resume_training=True):
    """
    Fonction d'entraînement avec reprise automatique depuis un checkpoint[6][11].
    """

    print("🚀 DÉBUT DE L'ENTRAÎNEMENT AVEC CHECKPOINTS")
    print("=" * 80)

    # Initialisation du gestionnaire de checkpoints
    checkpoint_manager = EnhancedCheckpointManager(
        checkpoint_dir=checkpoint_dir,
        model_name="pose_classification_model",
        save_top_k=5
    )

    # Configuration de l'entraînement
    group_criterion = nn.CrossEntropyLoss()
    exercise_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

    # Tentative de reprise depuis un checkpoint existant[6]
    start_epoch = 0
    best_val_exercise_acc = 0.0

    if resume_training and checkpoint_manager.check_for_existing_checkpoint():
        print("🔄 Checkpoint existant détecté. Reprise de l'entraînement...")
        start_epoch = checkpoint_manager.load_checkpoint(
            model, optimizer, scheduler, load_best=False
        )
        best_val_exercise_acc = checkpoint_manager.best_score
        print(f"📈 Reprise depuis l'époque {start_epoch + 1}")
    else:
        print("🆕 Démarrage d'un nouvel entraînement")

    # Boucle d'entraînement avec reprise correcte[6]
    for epoch in range(start_epoch, num_epochs):
        # Phase d'entraînement
        model.train()
        train_metrics = {
            'loss': 0.0,
            'group_loss': 0.0,
            'exercise_loss': 0.0,
            'group_acc': 0.0,
            'exercise_acc': 0.0,
            'total_samples': 0
        }

        correct_groups_train = 0
        correct_exercises_train = 0

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]") as pbar:
            for batch in pbar:
                if batch is None:
                    continue

                sequences, exercise_labels, group_labels = batch
                sequences = sequences.to(device)
                exercise_labels = exercise_labels.to(device)
                group_labels = group_labels.to(device)

                optimizer.zero_grad()

                group_outputs, exercise_outputs = model(sequences)

                group_loss = group_criterion(group_outputs, group_labels)
                exercise_loss = exercise_criterion(exercise_outputs, exercise_labels)
                total_loss = group_loss + exercise_loss

                total_loss.backward()
                optimizer.step()
                scheduler.step()

                # Métriques
                batch_size = sequences.size(0)
                train_metrics['loss'] += total_loss.item() * batch_size
                train_metrics['group_loss'] += group_loss.item() * batch_size
                train_metrics['exercise_loss'] += exercise_loss.item() * batch_size
                train_metrics['total_samples'] += batch_size

                # Accuracy
                _, predicted_groups = torch.max(group_outputs, 1)
                correct_groups_train += (predicted_groups == group_labels).sum().item()

                _, predicted_exercises = torch.max(exercise_outputs, 1)
                correct_exercises_train += (predicted_exercises == exercise_labels).sum().item()

                pbar.set_postfix({
                    'loss': train_metrics['loss'] / train_metrics['total_samples'],
                    'lr': scheduler.get_last_lr()[0]
                })

        # Finaliser les métriques d'entraînement
        if train_metrics['total_samples'] > 0:
            train_metrics['loss'] /= train_metrics['total_samples']
            train_metrics['group_loss'] /= train_metrics['total_samples']
            train_metrics['exercise_loss'] /= train_metrics['total_samples']
            train_metrics['group_acc'] = correct_groups_train / train_metrics['total_samples']
            train_metrics['exercise_acc'] = correct_exercises_train / train_metrics['total_samples']

        # Phase de validation
        model.eval()
        val_metrics = {
            'loss': 0.0,
            'group_loss': 0.0,
            'exercise_loss': 0.0,
            'group_acc': 0.0,
            'exercise_acc': 0.0,
            'total_samples': 0
        }

        correct_groups_val = 0
        correct_exercises_val = 0

        with torch.no_grad():
            with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]") as pbar:
                for batch in pbar:
                    if batch is None:
                        continue

                    sequences, exercise_labels, group_labels = batch
                    sequences = sequences.to(device)
                    exercise_labels = exercise_labels.to(device)
                    group_labels = group_labels.to(device)

                    group_outputs, exercise_outputs = model(sequences)

                    group_loss = group_criterion(group_outputs, group_labels)
                    exercise_loss = exercise_criterion(exercise_outputs, exercise_labels)
                    total_loss = group_loss + exercise_loss

                    batch_size = sequences.size(0)
                    val_metrics['loss'] += total_loss.item() * batch_size
                    val_metrics['group_loss'] += group_loss.item() * batch_size
                    val_metrics['exercise_loss'] += exercise_loss.item() * batch_size
                    val_metrics['total_samples'] += batch_size

                    # Accuracy
                    _, predicted_groups = torch.max(group_outputs, 1)
                    correct_groups_val += (predicted_groups == group_labels).sum().item()

                    _, predicted_exercises = torch.max(exercise_outputs, 1)
                    correct_exercises_val += (predicted_exercises == exercise_labels).sum().item()

                    pbar.set_postfix({
                        'loss': val_metrics['loss'] / val_metrics['total_samples'],
                        'group_acc': correct_groups_val / val_metrics['total_samples'] * 100,
                        'exercise_acc': correct_exercises_val / val_metrics['total_samples'] * 100
                    })

        # Finaliser les métriques de validation
        if val_metrics['total_samples'] > 0:
            val_metrics['loss'] /= val_metrics['total_samples']
            val_metrics['group_loss'] /= val_metrics['total_samples']
            val_metrics['exercise_loss'] /= val_metrics['total_samples']
            val_metrics['group_acc'] = correct_groups_val / val_metrics['total_samples']
            val_metrics['exercise_acc'] = correct_exercises_val / val_metrics['total_samples']

        # Affichage des résultats
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train - Loss: {train_metrics['loss']:.4f}, Group Acc: {train_metrics['group_acc']:.2%}, Exercise Acc: {train_metrics['exercise_acc']:.2%}")
        print(f"  Val   - Loss: {val_metrics['loss']:.4f}, Group Acc: {val_metrics['group_acc']:.2%}, Exercise Acc: {val_metrics['exercise_acc']:.2%}")

        # Sauvegarde des checkpoints[11]
        is_best = val_metrics['exercise_acc'] > best_val_exercise_acc
        if is_best:
            best_val_exercise_acc = val_metrics['exercise_acc']

        checkpoint_manager.save_checkpoint(
            model=model,
            optimizer=optimizer,
            scheduler=scheduler,
            epoch=epoch,
            train_metrics=train_metrics,
            val_metrics=val_metrics,
            is_best=is_best
        )

        print("-" * 80)

    print(f"\n🎉 Entraînement terminé!")
    print(f"🏆 Meilleure précision d'exercice: {best_val_exercise_acc:.2%}")

    return checkpoint_manager

# ========================================================
# PIPELINE PRINCIPAL ÉPURÉ
# ========================================================

def complete_training_pipeline_clean(data_dir, exercises_points, group_mapping,
                                   checkpoint_dir='/content/drive/MyDrive/checkpoints_enhanced',
                                   device='cuda', resume_training=True):
    """
    Pipeline d'entraînement épuré sans validation/nettoyage des données[15].
    """
    print("🚀 PIPELINE D'ENTRAÎNEMENT ÉPURÉ AVEC REPRISE AUTOMATIQUE")
    print("=" * 80)

    # Chargement du dataset (supposé déjà nettoyé)
    print("📂 Chargement du dataset préparé...")
    full_dataset = EnhancedExerciseHierarchicalDataset(
        data_dir=data_dir,
        exercices_points=exercises_points,
        grouped_sports=group_mapping,
        max_seq_len=50,
        augment=True,
        validate_physics=True
    )

    if len(full_dataset) == 0:
        print("❌ Erreur: Aucun échantillon valide chargé.")
        return None

    print(f"✅ Dataset chargé: {len(full_dataset)} échantillons")

    # Configuration du modèle
    input_dim = full_dataset.max_feature_dim
    num_groups = len(full_dataset.group_to_idx)
    num_exercises = len(full_dataset.exercise_to_idx)
    hidden_dim = 128

    print(f"📊 Configuration du modèle:")
    print(f"   Input dim: {input_dim}")
    print(f"   Hidden dim: {hidden_dim}")
    print(f"   Groupes: {num_groups}")
    print(f"   Exercices: {num_exercises}")

    # Instanciation du modèle
    model = PoseClassificationModel(input_dim, hidden_dim, num_groups, num_exercises).to(device)

    # Division du dataset
    total_size = len(full_dataset)
    train_size = int(0.7 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, [train_size, val_size, test_size]
    )

    print(f"📊 Division du dataset: Train({len(train_dataset)}), Val({len(val_dataset)}), Test({len(test_dataset)})")

    # Fonction de collation robuste[15]
    def collate_fn(batch):
        batch = list(filter(lambda x: x is not None, batch))
        if not batch:
            return None
        sequences, labels, groups = zip(*batch)
        padded_sequences = torch.stack(sequences)
        return padded_sequences, torch.stack(labels), torch.stack(groups)

    # DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             collate_fn=collate_fn, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                           collate_fn=collate_fn, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            collate_fn=collate_fn, num_workers=2)

    # Sauvegarder les mappings
    mappings_to_save = {
        'exercise_points': exercises_points,
        'group_mapping': group_mapping,
        'exercise_to_idx': full_dataset.exercise_to_idx,
        'idx_to_exercise': full_dataset.idx_to_exercise,
        'group_to_idx': full_dataset.group_to_idx,
        'idx_to_group': full_dataset.idx_to_group,
        'input_dim': input_dim,
        'hidden_dim': hidden_dim,
        'num_groups': num_groups,
        'num_exercises': num_exercises
    }

    mapping_path = os.path.join(checkpoint_dir, 'model_mappings.json')
    os.makedirs(checkpoint_dir, exist_ok=True)
    with open(mapping_path, 'w') as f:
        json.dump(mappings_to_save, f, indent=4, default=str)
    print(f"💾 Mappings sauvegardés: {mapping_path}")

    # Entraînement avec reprise automatique[6]
    checkpoint_manager = train_model_with_resume(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=50,
        checkpoint_dir=checkpoint_dir,
        device=device,
        resume_training=resume_training
    )

    return checkpoint_manager, model, (train_loader, val_loader, test_loader)

# ========================================================
# UTILISATION PRINCIPALE
# ========================================================

if __name__ == "__main__":
    # Configuration
    DATA_DIR = '/content/drive/MyDrive/preprocessed_videos_enhanced'
    CHECKPOINT_DIR = '/content/drive/MyDrive/checkpoints_enhanced'

    # Lancement du pipeline épuré avec reprise automatique
    try:
        results = complete_training_pipeline_clean(
            data_dir=DATA_DIR,
            exercises_points=exercises_points,  # Défini dans votre code précédent
            group_mapping=group_mapping,        # Défini dans votre code précédent
            checkpoint_dir=CHECKPOINT_DIR,
            device=device,  # Défini dans votre code précédent
            resume_training=True  # Active la reprise automatique
        )

        if results:
            checkpoint_manager, trained_model, data_loaders = results
            print("\n✅ Pipeline d'entraînement terminé avec succès!")

            # Accès aux résultats
            best_checkpoints = checkpoint_manager.get_best_checkpoints_summary()
            print("🏆 Meilleurs modèles:")
            for i, cp in enumerate(best_checkpoints[:3]):
                print(f"  {i+1}. Epoch {cp['epoch']}: {cp['val_exercise_acc']:.2%} exercise acc")
        else:
            print("\n❌ Échec du pipeline d'entraînement.")

    except Exception as e:
        print(f"\n💥 Erreur lors de l'exécution: {e}")
        import traceback
        traceback.print_exc()




🚀 PIPELINE D'ENTRAÎNEMENT ÉPURÉ AVEC REPRISE AUTOMATIQUE
📂 Chargement du dataset préparé...
Scanning directory for preprocessed data: /content/drive/MyDrive/preprocessed_videos_enhanced
Found 22 present exercises with data and group mapping.
exercise_to_idx (dynamic): {'barbell biceps curl': 0, 'bench press': 1, 'chest fly machine': 2, 'deadlift': 3, 'decline bench press': 4, 'hammer curl': 5, 'hip thrust': 6, 'incline bench press': 7, 'lat pulldown': 8, 'lateral raise': 9, 'leg extension': 10, 'leg raises': 11, 'plank': 12, 'pull up': 13, 'push-up': 14, 'romanian deadlift': 15, 'russian twist': 16, 'shoulder press': 17, 'squat': 18, 't bar row': 19, 'tricep dips': 20, 'tricep pushdown': 21}
exercise_to_group (dynamic): {'bench press': 'horizontal pushes', 'incline bench press': 'horizontal pushes', 'push-up': 'horizontal pushes', 'shoulder press': 'vertical pushes', 'tricep dips': 'vertical pushes', 'tricep pushdown': 'vertical pushes', 'lateral raise': 'vertical pushes', 't bar row':

Epoch 1/50 [Train]:   0%|          | 0/71 [00:00<?, ?it/s]

Epoch 1/50 [Train]:   1%|▏         | 1/71 [00:00<00:44,  1.57it/s, loss=5.06, lr=0.001]

Epoch 1/50 [Train]:   4%|▍         | 3/71 [00:01<00:24,  2.77it/s, loss=5.05, lr=0.001]

Epoch 1/50 [Train]:   4%|▍         | 3/71 [00:01<00:24,  2.77it/s, loss=5.04, lr=0.001]

...

Epoch 4/50 [Train]:  85%|████████▍ | 60/71 [00:30<00:03,  3.26it/s, loss=1.64, lr=0.0001]

Epoch 4/50 [Train]:  89%|████████▊ | 63/71 [00:31<00:02,  3.39it/s, loss=1.64, lr=0.0001]

Epoch 4/50 [Train]:  92%|█████████▏| 65/71 [00:31<00:02,  2.95it/s, loss=1.65, lr=0.0001]

Epoch 4/50 [Train]:  94%|█████████▍| 67/71 [00:32<00:01,  2.79it/s, loss=1.65, lr=0.0001]

Epoch 4/50 [Train]:  96%|█████████▌| 68/71 [00:33<00:01,  2.76it/s, loss=1.66, lr=0.0001]

In [1]:
# prompt: charger le modèle et tester le sur les 10% non utiliser

# --- Charger le modèle et tester sur l'ensemble de test ---

# Charger le modèle avec les mêmes dimensions que lors de l'entraînement
# Assurez-vous que `input_dim`, `hidden_dim`, `num_groups`, `num_exercises`
# correspondent à ceux utilisés pour entraîner le modèle sauvegardé.
# Une façon de le faire est de recharger les mappings.

full_dataset = EnhancedExerciseHierarchicalDataset(
    data_dir=dataset_config['data_dir'],
    exercices_points=exercises_points, # Assuming exercises_points is available
    grouped_sports=group_mapping,     # Assuming group_mapping is available
    max_seq_len=dataset_config['max_seq_len'],
    augment=dataset_config['augment'],
    validate_physics=dataset_config['validate_physics']
)

try:
    with open('/content/drive/MyDrive/pose_classification_mappings1.json', 'r') as f:
        loaded_mappings = json.load(f)

    # Reconstituer les mappings et les dimensions du modèle
    loaded_exercise_to_idx = loaded_mappings['exercise_to_idx']
    loaded_group_to_idx = loaded_mappings['group_to_idx']
    loaded_exercises_points = loaded_mappings['exercise_points'] # Use loaded points for dim calc
    loaded_group_mapping = loaded_mappings['group_mapping']

    # Calculer l'input_dim à partir des mappings chargés
    # This calculation must match the one in the EnhancedExerciseHierarchicalDataset
    def calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercises_points):
        max_dim = 0
        # Iterate through exercises that were actually used in the dataset
        for exercise_name in loaded_exercise_to_idx.keys():
             if exercise_name in loaded_exercises_points:
                num_points = len(loaded_exercises_points[exercise_name])
                # Coordonnées 3D + Angles + Distances + Vitesses + Accélérations
                current_dim = num_points * 3 + 3 + 2 + num_points * 3 + num_points * 3
                max_dim = max(max_dim, current_dim)
        return max_dim if max_dim > 0 else 0

    loaded_input_dim = calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercises_points)
    loaded_num_groups = len(loaded_group_to_idx)
    loaded_num_exercises = len(loaded_exercise_to_idx) # Number of exercises actually in the dataset

    print("\n--- Charger le modèle pour le test ---")
    print(f"Dimensions du modèle chargé (calculées depuis mappings):")
    print(f"  Input Dim: {loaded_input_dim}")
    print(f"  Nombre de Groupes: {loaded_num_groups}")
    print(f"  Nombre d'Exercices: {loaded_num_exercises}")


    # Instancier le modèle avec les dimensions chargées
    test_model = PoseClassificationModel(loaded_input_dim, hidden_dim, loaded_num_groups, loaded_num_exercises).to(device)

    # Charger les poids du meilleur modèle entraîné
    best_model_path = '/content/drive/MyDrive/best_pose_classification_model1.pth'
    if os.path.exists(best_model_path):
        test_model.load_state_dict(torch.load(best_model_path))
        print(f"Modèle chargé depuis {best_model_path}")
    else:
        print(f"Erreur: Fichier modèle non trouvé à {best_model_path}. L'entraînement a-t-il réussi?")
        # Exit or handle the error appropriately if the model cannot be loaded
        # For this example, we'll stop here.
        exit() # or raise Exception("Model file not found")

    test_model.eval() # Mettre le modèle en mode évaluation

    # Utiliser le test_loader déjà créé (qui correspond aux 10% non utilisés)
    if 'test_loader' in locals() and len(test_dataset) > 0:
        print("\nDébut de l'évaluation sur l'ensemble de test final...")

        test_running_loss = 0.0
        test_running_group_loss = 0.0
        test_running_exercise_loss = 0.0
        test_correct_groups = 0
        test_correct_exercises = 0
        total_test_samples = 0

        # Criteria for calculating loss on test set
        test_group_criterion = nn.CrossEntropyLoss()
        test_exercise_criterion = nn.CrossEntropyLoss()


        with torch.no_grad(): # Désactiver le calcul des gradients
            with tqdm(test_loader, desc="[Final Test Evaluation]") as pbar:
                for batch in pbar:
                    if batch is None: # Skip empty batches
                        continue
                    sequences, exercise_labels, group_labels = batch

                    group_outputs, exercise_outputs = test_model(sequences)

                    group_loss = test_group_criterion(group_outputs, group_labels)
                    exercise_loss = test_exercise_criterion(exercise_outputs, exercise_labels)

                    total_loss = group_loss + exercise_loss

                    test_running_loss += total_loss.item() * sequences.size(0)
                    test_running_group_loss += group_loss.item() * sequences.size(0)
                    test_running_exercise_loss += exercise_loss.item() * sequences.size(0)
                    total_test_samples += sequences.size(0)

                    # Calcul de l'accuracy
                    _, predicted_groups = torch.max(group_outputs, 1)
                    test_correct_groups += (predicted_groups == group_labels).sum().item()

                    _, predicted_exercises = torch.max(exercise_outputs, 1)
                    test_correct_exercises += (predicted_exercises == exercise_labels).sum().item()

                    # Update progress bar description
                    pbar.set_postfix({
                         'loss': test_running_loss / total_test_samples if total_test_samples > 0 else 0.0,
                         'group_acc': test_correct_groups / total_test_samples * 100 if total_test_samples > 0 else 0.0,
                         'exercise_acc': test_correct_exercises / total_test_samples * 100 if total_test_samples > 0 else 0.0
                     })


        test_epoch_loss = test_running_loss / total_test_samples if total_test_samples > 0 else 0
        test_epoch_group_loss = test_running_group_loss / total_test_samples if total_test_samples > 0 else 0
        test_epoch_exercise_loss = test_running_exercise_loss / total_test_samples if total_test_samples > 0 else 0
        test_group_accuracy = test_correct_groups / total_test_samples if total_test_samples > 0 else 0
        test_exercise_accuracy = test_correct_exercises / total_test_samples if total_test_samples > 0 else 0

        print(f"\nRésultats FINAUX sur l'ensemble de test (10% non utilisé):")
        print(f"  Test Loss: {test_epoch_loss:.4f} (Group: {test_epoch_group_loss:.4f}, Exercise: {test_epoch_exercise_loss:.4f})")
        print(f"  Test Accuracy: Group: {test_group_accuracy:.2%}, Exercise: {test_exercise_accuracy:.2%}") # Report as percentage

    else:
        print("\nPas d'ensemble de test disponible pour l'évaluation finale. L'entraînement a-t-il généré un test_loader?")


except FileNotFoundError:
    print("\nErreur: Fichier mappings.json non trouvé. L'entraînement a-t-il été exécuté pour créer ce fichier?")
except Exception as e:
    print(f"\nUne erreur s'est produite lors du chargement ou du test du modèle: {str(e)}")

print("\nFin du processus de test du modèle.")


NameError: name 'EnhancedExerciseHierarchicalDataset' is not defined

In [11]:
# prompt: réecriver le modèle et importer le pth puis utilise le pth pour créer les groupes et exercice et cette fois on va tester sur les vidéos des exercices qui connais et voir la précision elle doit être totalement indépendante cette cellule exercice point et groupe doivent être prisent du pth pas écrite

# Assurez-vous que les bibliothèques nécessaires sont importées
import cv2
import mediapipe as mp_lib
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import torch.nn as nn
import random
import torch.optim as optim
import json # To save mappings
import warnings
warnings.filterwarnings('ignore')

# Assurez-vous que 'device' est défini (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Configuration et Mappings (Doivent correspondre à ceux utilisés pour l'entraînement) ---

# Définir les chemins des fichiers sauvegardés lors de l'entraînement
best_model_path = '/content/drive/MyDrive/best_pose_classification_model1.pth'
mapping_path = '/content/drive/MyDrive/pose_classification_mappings1.json'
data_dir = '/content/drive/MyDrive/preprocessed_videos_enhanced' # Répertoire des données prétraitées
max_seq_len = 50 # Doit correspondre au max_seq_len utilisé lors de l'entraînement

# Charger les mappings et la configuration utilisés lors de l'entraînement
try:
    with open(mapping_path, 'r') as f:
        loaded_mappings = json.load(f)

    loaded_exercise_points = loaded_mappings['exercise_points']
    loaded_group_mapping = loaded_mappings['group_mapping']
    loaded_exercise_to_idx = loaded_mappings['exercise_to_idx']
    loaded_idx_to_exercise = {int(k): v for k, v in loaded_mappings['idx_to_exercise'].items()} # Convert keys to int
    loaded_group_to_idx = loaded_mappings['group_to_idx']
    loaded_idx_to_group = {int(k): v for k, v in loaded_mappings['idx_to_group'].items()} # Convert keys to int


    # Recalculer la dimension d'entrée en utilisant les points clés des mappings chargés
    def calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercise_points):
        max_dim = 0
        # Iterate through exercises that were actually used in the dataset
        for exercise_name in loaded_exercise_to_idx.keys():
             if exercise_name in loaded_exercise_points:
                num_points = len(loaded_exercise_points[exercise_name])
                # Coordonnées 3D + Angles + Distances + Vitesses + Accélérations
                # This must match the feature calculation in EnhancedExercisePreprocessor
                base_features = num_points * 3
                angles = 3
                distances = 2
                dynamics = base_features * 2
                current_dim = base_features + angles + distances + dynamics
                max_dim = max(max_dim, current_dim)
        return max_dim if max_dim > 0 else 0

    loaded_input_dim = calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercise_points)
    loaded_num_groups = len(loaded_group_to_idx)
    loaded_num_exercises = len(loaded_exercise_to_idx)

    print("Mappings et configuration chargés.")
    print(f"  Input Dim: {loaded_input_dim}")
    print(f"  Nombre de Groupes: {loaded_num_groups}")
    print(f"  Nombre d'Exercices: {loaded_num_exercises}")

except FileNotFoundError:
    print(f"Erreur: Fichier mappings.json non trouvé à {mapping_path}. Assurez-vous que l'entraînement a été exécuté.")
    # Vous pouvez choisir d'arrêter ici si les mappings sont essentiels
    exit()
except Exception as e:
    print(f"Erreur lors du chargement des mappings: {str(e)}")
    exit()


# --- Définition de la classe Modèle (Doit être identique à celle utilisée pour l'entraînement) ---

class PoseClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_groups, num_exercises):
        super(PoseClassificationModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_groups = num_groups
        self.num_exercises = num_exercises

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        self.group_classifier = nn.Linear(hidden_dim, num_groups)

        self.exercise_classifier = nn.Linear(hidden_dim, num_exercises)

    def forward(self, sequence):
        out, (hn, cn) = self.lstm(sequence)
        final_hidden_state = hn[-1, :, :] # Get the hidden state of the last layer
        group_output = self.group_classifier(final_hidden_state)
        exercise_output = self.exercise_classifier(final_hidden_state)
        return group_output, exercise_output

# --- Définition de la classe Dataset pour les vidéos de test (Doit être similaire au Dataset d'entraînement mais sans augmentation et avec un chemin spécifique) ---

class EnhancedVideoPredictDataset(Dataset):
    """Dataset pour charger et prétraiter des vidéos uniques pour la prédiction."""
    def __init__(self, video_paths, exercises_points, group_mapping, max_seq_len,
                 use_physics_validation=True, use_temporal_smoothing=True):
        self.video_paths = video_paths
        self.max_seq_len = max_seq_len
        self.exercises_points = exercises_points
        self.group_mapping = group_mapping # Needed to find the exercise name from group mapping

        # Re-instantiate the preprocessor with the correct max_frames (max_seq_len)
        # and ensure other settings match training if needed.
        self.preprocessor = EnhancedExercisePreprocessor(
            max_frames=max_seq_len,
            step=2, # Use the same step as training preprocessor
            use_physics_validation=use_physics_validation,
            use_temporal_smoothing=use_temporal_smoothing
        )


        # Determine max_feature_dim from loaded mappings for consistent padding
        # This assumes loaded_exercise_to_idx and loaded_exercise_points are available in the outer scope
        self.max_feature_dim = calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercise_points)


    def __len__(self):
        return len(self.video_paths)

    def _extract_exercise_name_from_path(self, video_path):
        """Tente d'extraire le nom d'exercice du chemin du fichier."""
        path_parts = Path(video_path).parts
        # Parcourir les parties du chemin à l'envers pour trouver le nom de l'exercice
        # The exercise name is assumed to be the parent directory name within data_dir
        data_dir_path = Path(data_dir) # Assuming data_dir is available in outer scope
        try:
             relative_path = Path(video_path).relative_to(data_dir_path)
             # The first part of the relative path should be the exercise name
             if len(relative_path.parts) > 1:
                 exercise_name = relative_path.parts[0]
                 if exercise_name in self.exercises_points:
                     return exercise_name
        except ValueError:
             # video_path is not relative to data_dir_path, try other methods
             pass


        for part in reversed(path_parts):
            if part in self.exercises_points:
                return part
            # Alternativement, si le nom de l'exercice peut être le nom du dossier parent
            for group_name, exercises in self.group_mapping.items():
                 if part in exercises:
                      return part # Assuming folder name is the exercise name


        print(f"Warning: Could not determine exercise name from path: {video_path}. Using 'unknown'.")
        return "unknown" # Fallback

    def _standardize_sequence(self, seq, target_feature_dim):
        """Standardize sequence length and feature dimension for prediction."""
         # Feature adjustment
        if seq.shape[1] < target_feature_dim:
            padding = target_feature_dim - seq.shape[1]
            seq = torch.nn.functional.pad(seq, (0, padding), mode='constant', value=0)
        elif seq.shape[1] > target_feature_dim:
            seq = seq[:, :target_feature_dim]

        # Temporal adjustment
        if seq.shape[0] > self.max_seq_len:
            # If processing video gives more frames than max_seq_len, truncate
            seq = seq[:self.max_seq_len]
        elif seq.shape[0] < self.max_seq_len:
             # If processing video gives fewer frames, pad
            padding = self.max_seq_len - seq.shape[0]
            seq = torch.nn.functional.pad(seq, (0, 0, 0, padding), mode='constant', value=0)


        return seq


    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        # Try to extract exercise name from path
        # For this prediction dataset, we assume the video file is a .npy from preprocessing
        # The exercise name is the parent directory.
        exercise_name = Path(video_path).parent.name

        if exercise_name not in self.exercises_points:
             print(f"Skipping video {video_path}: Parent directory '{exercise_name}' is not a known exercise.")
             return None

        # print(f"Loading preprocessed data for prediction: {Path(video_path).name} (Assumed exercise: {exercise_name})")


        try:
            # Load the numpy array directly from the .npy file
            seq_np = np.load(video_path, mmap_mode='r', allow_pickle=False)

            # Ensure the data is float32 and handle potential non-finite values after loading
            seq_np = seq_np.astype(np.float32)
            if np.isnan(seq_np).any() or np.isinf(seq_np).any():
                 print(f"Warning: NaN or Inf found in {video_path} after loading. Skipping.")
                 return None # Skip this sample

            seq = torch.from_numpy(seq_np)

            # Standardize the sequence to match model's expected dimensions
            seq = self._standardize_sequence(seq, self.max_feature_dim)

            # For prediction, we just need the sequence and the path for reporting
            return seq.to(device), video_path # Move to device here


        except Exception as e:
            print(f"❌ Error loading or processing {video_path}: {str(e)}")
            return None


# --- Fonction d'évaluation sur des vidéos de test (nouvelle cellule) ---

def predict_on_test_videos(model, data_dir, loaded_exercise_to_idx, loaded_idx_to_exercise, loaded_group_to_idx, loaded_idx_to_group,
                          loaded_exercise_points, loaded_group_mapping, max_seq_len, num_videos_per_exercise=3):
    """
    Évalue le modèle sur un échantillon aléatoire de vidéos prétraitées pour chaque exercice
    présent dans les mappings chargés.

    Args:
        model (nn.Module): Le modèle PyTorch entraîné.
        data_dir (str): Chemin vers le répertoire contenant les données prétraitées (.npy files).
        loaded_exercise_to_idx (dict): Mapping des noms d'exercices aux indices chargé.
        loaded_idx_to_exercise (dict): Mapping des indices aux noms d'exercices chargé.
        loaded_group_to_idx (dict): Mapping des noms de groupes aux indices chargé.
        loaded_idx_to_group (dict): Mapping des indices aux noms de groupes chargé.
        loaded_exercise_points (dict): Dictionnaire des points clés par exercice chargé.
        loaded_group_mapping (dict): Dictionnaire des exercices groupés chargé.
        max_seq_len (int): La longueur maximale des séquences attendue par le modèle.
        num_videos_per_exercise (int): Nombre de vidéos aléatoires à sélectionner par exercice.
    """
    print(f"\n--- Début de la prédiction sur {num_videos_per_exercise} vidéos aléatoires par exercice ---")

    # Collecter les chemins des fichiers .npy pour les exercices présents dans les mappings
    video_files_to_test = []
    for exercise_name in loaded_exercise_to_idx.keys(): # Iterate only over exercises the model knows
        exercise_dir = Path(data_dir) / exercise_name
        if exercise_dir.exists() and exercise_dir.is_dir():
            npy_files = list(exercise_dir.glob('*.npy'))
            if npy_files:
                # Sélectionnez aléatoirement num_videos_per_exercise fichiers par exercice
                selected_files = random.sample(npy_files, min(num_videos_per_exercise, len(npy_files)))
                video_files_to_test.extend(selected_files)
                print(f"  Sélectionné {len(selected_files)} fichiers pour {exercise_name}")
            else:
                 print(f"  Aucun fichier .npy trouvé pour {exercise_name}")

    if not video_files_to_test:
        print("Aucun fichier prétraité trouvé pour les exercices connus du modèle.")
        return

    print(f"Total de {len(video_files_to_test)} fichiers sélectionnés pour la prédiction.")


    # Créer un Dataset et un DataLoader spécifiques pour la prédiction
    predict_dataset = EnhancedVideoPredictDataset(
        video_paths=[str(p) for p in video_files_to_test], # Convertir Path en str
        exercises_points=loaded_exercise_points,
        group_mapping=loaded_group_mapping,
        max_seq_len=max_seq_len,
        use_physics_validation=True, # Use validation during prediction processing
        use_temporal_smoothing=True # Use smoothing during prediction processing
    )

    # Custom collate_fn for the prediction dataset
    def predict_collate_fn(batch):
         # Filter out None samples (failed processing)
         batch = list(filter(lambda x: x is not None, batch))
         if not batch:
              return None
         # Batch structure: [(processed_sequence1, video_path1), (processed_sequence2, video_path2), ...]
         sequences, video_paths = zip(*batch)
         # sequences are already padded in __getitem__ and moved to device
         padded_sequences = torch.stack(sequences)
         return padded_sequences, video_paths # Return the list of video paths


    predict_loader = DataLoader(predict_dataset, batch_size=1, shuffle=False, collate_fn=predict_collate_fn) # Batch size 1 for individual video processing

    model.eval() # Mettre le modèle en mode évaluation

    predictions = {}

    with torch.no_grad():
        with tqdm(predict_loader, desc="[Predicting]") as pbar:
            for batch_data in pbar:
                if batch_data is None: # Skip empty batches from collate_fn
                     continue

                # batch_data is a tuple: (sequences, video_paths)
                sequences, video_paths = batch_data

                # sequences is a batch (batch_size=1), video_paths is a tuple of paths
                sequence = sequences[0].unsqueeze(0) # Get the single sequence and add batch dim back
                video_path = video_paths[0] # Get the single video path


                group_outputs, exercise_outputs = model(sequence)

                # Obtenir les prédictions (indices de classe)
                _, predicted_group_idx = torch.max(group_outputs, 1)
                _, predicted_exercise_idx = torch.max(exercise_outputs, 1)

                # Convertir les indices en noms en utilisant les mappings chargés
                predicted_group_name = loaded_idx_to_group.get(predicted_group_idx.item(), "Unknown Group")
                predicted_exercise_name = loaded_idx_to_exercise.get(predicted_exercise_idx.item(), "Unknown Exercise")

                predictions[str(video_path)] = {
                    'group': predicted_group_name,
                    'exercise': predicted_exercise_name,
                    # Optionally include confidence scores
                    'group_confidence': torch.softmax(group_outputs, dim=1).max().item(),
                    'exercise_confidence': torch.softmax(exercise_outputs, dim=1).max().item()
                }

                # Update progress bar description
                pbar.set_postfix({'video': Path(video_path).name, 'predicted': predicted_exercise_name})


    print("\n--- Résultats des prédictions ---")
    correct_predictions = 0
    total_predictions = 0
    for video_path, preds in predictions.items():
        print(f"\nVidéo: {Path(video_path).name}")
        if preds:
            print(f"  Prédiction: Groupe - {preds['group']}, Exercice - {preds['exercise']}")
            # print(f"  Confiance: Groupe - {preds['group_confidence']:.2f}, Exercice - {preds['exercise_confidence']:.2f}") # Optional

            # Determine ground truth from the directory name
            true_exercise = Path(video_path).parent.name
            print(f"  Vérité terrain: {true_exercise}")

            # Check if prediction is correct
            if preds['exercise'] == true_exercise:
                 correct_predictions += 1
                 print("  ✅ Correct")
            else:
                 print("  ❌ Incorrect")
            total_predictions += 1

        else:
            print("  Aucune prédiction valide pour cette vidéo.")

    # Calculate and print overall accuracy
    overall_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
    print(f"\nPrécision globale sur les vidéos de test sélectionnées: {overall_accuracy:.2%}")


    print("\n--- Fin de la prédiction ---")


# --- Exécution de la prédiction ---

# Assurez-vous que le modèle est chargé avant d'appeler la fonction de prédiction
if 'test_model' in locals() and 'loaded_idx_to_exercise' in locals():
     predict_on_test_videos(
         model=test_model,
         data_dir=data_dir, # Use the loaded data_dir
         loaded_exercise_to_idx=loaded_exercise_to_idx,
         loaded_idx_to_exercise=loaded_idx_to_exercise,
         loaded_group_to_idx=loaded_group_to_idx,
         loaded_idx_to_group=loaded_idx_to_group,
         loaded_exercise_points=loaded_exercise_points,
         loaded_group_mapping=loaded_group_mapping,
         max_seq_len=max_seq_len, # Utilisez le max_seq_len chargé/défini
         num_videos_per_exercise=3 # Nombre de vidéos à tester par exercice
     )
else:
    print("Erreur: Le modèle ou les mappings n'ont pas été chargés correctement dans les cellules précédentes.")

Mappings et configuration chargés.
  Input Dim: 104
  Nombre de Groupes: 7
  Nombre d'Exercices: 22
Erreur: Le modèle ou les mappings n'ont pas été chargés correctement dans les cellules précédentes.


In [12]:
# prompt: donne moi un code similaire mais pour un environnement jupyter webcam exercice point et groupe sont pris du pth pas écrit dans la version donner le modèle et importation ne sont pas écrite

import cv2
import mediapipe as mp_lib
import numpy as np
from pathlib import Path
import torch
import torch.nn as nn
import random
import json
import time
import threading
from collections import deque
from tqdm import tqdm

# Ensure libraries are installed
try:
    import cv2
except ImportError:
    !pip install opencv-python
    import cv2

try:
    import mediapipe as mp_lib
except ImportError:
    !pip install mediapipe
    import mediapipe as mp_lib

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load mappings and configuration (adjust paths as necessary for your Jupyter environment)
# Ensure these paths exist and contain the files saved from your training code
mapping_path = 'pose_classification_mappings1.json' # Adjust path if needed
model_path = 'best_pose_classification_model1.pth' # Adjust path if needed

try:
    with open(mapping_path, 'r') as f:
        loaded_mappings = json.load(f)

    loaded_exercise_points = loaded_mappings['exercise_points']
    loaded_group_mapping = loaded_mappings['group_mapping']
    loaded_exercise_to_idx = loaded_mappings['exercise_to_idx']
    loaded_idx_to_exercise = {int(k): v for k, v in loaded_mappings['idx_to_exercise'].items()} # Convert keys to int
    loaded_group_to_idx = loaded_mappings['group_to_idx']
    loaded_idx_to_group = {int(k): v for k, v in loaded_mappings['idx_to_group'].items()} # Convert keys to int

    # Recalculate input dimension based on loaded mappings
    def calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercise_points):
        max_dim = 0
        for exercise_name in loaded_exercise_to_idx.keys():
             if exercise_name in loaded_exercise_points:
                num_points = len(loaded_exercise_points[exercise_name])
                # This must match the feature calculation in your preprocessing code
                base_features = num_points * 3 # XYZ
                angles = 3
                distances = 2
                dynamics = base_features * 2 # Velocity and Acceleration for each point
                current_dim = base_features + angles + distances + dynamics
                max_dim = max(max_dim, current_dim)
        return max_dim if max_dim > 0 else 0

    loaded_input_dim = calculate_loaded_input_dim(loaded_exercise_to_idx, loaded_exercise_points)
    loaded_num_groups = len(loaded_group_to_idx)
    loaded_num_exercises = len(loaded_exercise_to_idx)

    print("Mappings and configuration loaded for inference.")
    print(f"  Input Dim: {loaded_input_dim}")
    print(f"  Nombre de Groupes: {loaded_num_groups}")
    print(f"  Nombre d'Exercices: {loaded_num_exercises}")
    print(f"  Expected Max Seq Length: {loaded_mappings.get('max_seq_len', 'Unknown - Check training script')}") # If max_seq_len was saved

except FileNotFoundError:
    print(f"Error: Mappings or model file not found. Please ensure {mapping_path} and {model_path} exist.")
    # Exit or handle the error appropriately
    exit()
except Exception as e:
    print(f"Error loading mappings or model: {str(e)}")
    exit()

# --- Define the Model Class (Must be identical to the one used for training) ---
class PoseClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_groups, num_exercises):
        super(PoseClassificationModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_groups = num_groups
        self.num_exercises = num_exercises

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.group_classifier = nn.Linear(hidden_dim, num_groups)
        self.exercise_classifier = nn.Linear(hidden_dim, num_exercises)

    def forward(self, sequence):
        out, (hn, cn) = self.lstm(sequence)
        final_hidden_state = hn[-1, :, :]
        group_output = self.group_classifier(final_hidden_state)
        exercise_output = self.exercise_classifier(final_hidden_state)
        return group_output, exercise_output

# Instantiate and Load the Model
hidden_dim = 128 # Must match the hidden_dim used during training

model = PoseClassificationModel(loaded_input_dim, hidden_dim, loaded_num_groups, loaded_num_exercises).to(device)

if Path(model_path).exists():
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print(f"Model loaded successfully from {model_path}")
else:
    print(f"Error: Model file not found at {model_path}. Cannot proceed.")
    exit()

# --- MediaPipe Pose Setup ---
mp_pose = mp_lib.solutions.pose
mp_drawing = mp_lib.solutions.drawing_utils
mp_drawing_styles = mp_lib.solutions.drawing_styles

# Use the same config as preprocessing if possible
MP_CONFIG = {
    'static_image_mode': False,
    'model_complexity': 2,
    'min_detection_confidence': 0.5,
    'min_tracking_confidence': 0.5,
    'enable_segmentation': True,
    'smooth_landmarks': True,
    'smooth_segmentation': False
}

# --- Feature Extraction and Preprocessing (Simplified for real-time) ---

def extract_and_normalize_landmarks(results, frame_width, frame_height):
    """Extracts and normalizes 3D landmarks from MediaPipe results."""
    if not results.pose_world_landmarks:
        return None

    # Extract all 33 landmarks (x, y, z, visibility)
    landmarks = []
    for landmark in results.pose_world_landmarks.landmark:
         landmarks.extend([landmark.x, landmark.y, landmark.z]) # Use world landmarks

    # Convert to numpy array
    landmarks_np = np.array(landmarks, dtype=np.float32)

    # Basic normalization: Center around the hip or mid-point
    # Find a reference point (e.g., average of hips or mid-shoulder)
    # Using average of hips (landmarks 23 and 24) for centering
    hip_left = results.pose_world_landmarks.landmark[23]
    hip_right = results.pose_world_landmarks.landmark[24]
    center_x = (hip_left.x + hip_right.x) / 2
    center_y = (hip_left.y + hip_right.y) / 2
    center_z = (hip_left.z + hip_right.z) / 2

    center_point = np.array([center_x, center_y, center_z] * 33, dtype=np.float32)

    # Subtract the center point
    normalized_landmarks = landmarks_np - center_point

    # Optional: Scale by a reference distance (e.g., hip width)
    hip_distance = np.sqrt((hip_left.x - hip_right.x)**2 + (hip_left.y - hip_right.y)**2 + (hip_left.z - hip_right.z)**2)
    if hip_distance > 1e-6: # Avoid division by zero
         normalized_landmarks /= hip_distance

    # Now, filter to include only the points relevant to the loaded model's input dimension
    # We need to map the full 33 landmarks to the selected points for the exercises
    # This requires knowing which points the model was trained on for each exercise
    # Since the dataset class did this mapping, we need to replicate it here.
    # For simplicity, let's assume the model was trained on ALL 33 landmarks initially
    # and padding handled features not present in specific exercises.
    # A more robust approach would require knowing the exact point indices used for the model's input dimension.
    # Let's assume the loaded_input_dim corresponds to the expected features calculated from ALL 33 points
    # + angles + distances + dynamics.

    num_all_points = 33
    base_features_all = num_all_points * 3 # XYZ
    angles_all = 3 # Assuming generic angles like torso, elbow, knee for all 33 pts calculation
    distances_all = 2 # Assuming generic distances
    dynamics_all = base_features_all * 2 # Velocity and Acceleration

    expected_dim_if_all_points_processed = base_features_all + angles_all + distances_all + dynamics_all

    if normalized_landmarks.shape[0] * 3 != base_features_all:
         print(f"Warning: Landmark shape mismatch. Expected {base_features_all}, got {normalized_landmarks.shape[0]*3}. Using available data.")


    # Calculate additional features: angles, distances, velocity, acceleration
    # This requires a sequence of frames. For a single frame, we can only calculate instantaneous features or differentials if we have previous frames.
    # For real-time, we need a buffer of recent frames.

    # This function should ideally output the full feature vector (landmarks + angles + distances + dynamics)
    # matching the input_dim the model expects.
    # Since we only have landmarks here, we'll need to extend this.

    return normalized_landmarks # Return just normalized landmarks for now, will add dynamics later


def calculate_additional_features(sequence, exercises_points, max_seq_len, max_feature_dim):
    """
    Calculates angles, distances, velocities, and accelerations from a sequence of landmarks.
    Pads/truncates features to match max_feature_dim and max_seq_len.
    """
    # sequence: (num_frames, num_all_points * 3) numpy array
    num_frames, base_dim = sequence.shape
    num_all_points = base_dim // 3

    if num_frames == 0:
        return np.zeros((max_seq_len, max_feature_dim), dtype=np.float32)

    features_list = []

    for i in range(num_frames):
        frame_features = []
        current_landmarks = sequence[i].reshape(-1, 3) # Shape (num_all_points, 3)

        # 1. Base Normalized Landmarks (already included in sequence)
        frame_features.extend(current_landmarks.flatten().tolist())

        # 2. Angles (Example: Elbow, Knee, Torso Angle - Need specific landmark indices)
        # You need to map indices from 0-32 to calculate these.
        # Example: Right Elbow angle (landmarks 14, 12, 16)
        # Need to check if these points are included in the *original* Mediapipe 33 points.
        # Yes, they are.
        angles = []
        if num_all_points > 16: # Ensure indices exist
            # Right Elbow: 14 (wrist), 12 (shoulder), 16 (index finger) -> Should be 14, 12, 16? Let's use 14,12,16.
            # vector1 = p12 - p14, vector2 = p16 - p14. Angle between p12-p14 and p16-p14 is not elbow angle.
            # Elbow angle: between upper arm (12-14) and forearm (14-16). Angle between vector 14-12 and vector 14-16
            try:
                angle_re = calculate_angle_3d(current_landmarks[14], current_landmarks[12], current_landmarks[16]) # Point, Vertex, Point
            except IndexError:
                angle_re = 0.0
            # Left Elbow angle (13, 11, 15)
            try:
                angle_le = calculate_angle_3d(current_landmarks[13], current_landmarks[11], current_landmarks[15])
            except IndexError:
                angle_le = 0.0
            # Right Knee angle (26, 24, 28)
            try:
                angle_rk = calculate_angle_3d(current_landmarks[26], current_landmarks[24], current_landmarks[28])
            except IndexError:
                angle_rk = 0.0
            # Left Knee angle (25, 23, 27)
            try:
                angle_lk = calculate_angle_3d(current_landmarks[25], current_landmarks[23], current_landmarks[27])
            except IndexError:
                angle_lk = 0.0
            # Torso angle (angle between line joining hip center and shoulder center, and vertical axis)
            # Hip center: mid-point of 23 and 24
            # Shoulder center: mid-point of 11 and 12
            try:
                hip_center = (current_landmarks[23] + current_landmarks[24]) / 2
                shoulder_center = (current_landmarks[11] + current_landmarks[12]) / 2
                torso_vector = shoulder_center - hip_center
                vertical_vector = np.array([0, 1, 0], dtype=np.float32) # Assuming Y is up
                angle_torso = angle_between_vectors_3d(torso_vector, vertical_vector)
            except IndexError:
                 angle_torso = 0.0

            angles.extend([angle_re, angle_le, angle_rk, angle_lk, angle_torso])
        else:
             angles.extend([0.0] * 5) # Pad if not enough points

        frame_features.extend(angles)


        # 3. Distances (Example: Wrist-to-Hip, Elbow-to-Knee - Need specific landmark indices)
        distances = []
        if num_all_points > 28: # Ensure indices exist
            # Right Wrist to Right Hip (16 to 24)
            try:
                dist_rw_rh = np.linalg.norm(current_landmarks[16] - current_landmarks[24])
            except IndexError:
                dist_rw_rh = 0.0
            # Left Wrist to Left Hip (15 to 23)
            try:
                dist_lw_lh = np.linalg.norm(current_landmarks[15] - current_landmarks[23])
            except IndexError:
                dist_lw_lh = 0.0
            # Right Elbow to Right Knee (14 to 26)
            try:
                dist_re_rk = np.linalg.norm(current_landmarks[14] - current_landmarks[26])
            except IndexError:
                dist_re_rk = 0.0
            # Left Elbow to Left Knee (13 to 25)
            try:
                dist_le_lk = np.linalg.norm(current_landmarks[13] - current_landmarks[25])
            except IndexError:
                dist_le_lk = 0.0

            distances.extend([dist_rw_rh, dist_lw_lh, dist_re_rk, dist_le_lk])
        else:
             distances.extend([0.0] * 4) # Pad if not enough points

        frame_features.extend(distances)


        # 4. Dynamics (Velocity and Acceleration)
        # Requires previous frames. For frame i, velocity is (frame[i] - frame[i-1]) / time_diff
        # Acceleration is (velocity[i] - velocity[i-1]) / time_diff
        # For the first frame, velocity and acceleration are zero.

        velocities = []
        accelerations = []

        if i > 0:
            # Velocity: (current_landmarks - previous_landmarks) / time_diff (assuming constant frame rate, time_diff=1)
            # If using normalized landmarks, velocity is difference of normalized positions
            # Let's calculate dynamics based on the *base* normalized landmarks
            prev_landmarks = sequence[i-1].reshape(-1, 3)
            velocity = current_landmarks - prev_landmarks
            velocities.extend(velocity.flatten().tolist())

            if i > 1:
                 # Acceleration: (current_velocity - previous_velocity) / time_diff
                 # Need velocity from frame i-1. This approach is simplified.
                 # A proper preprocessor would buffer frames and calculate true derivatives.
                 # For real-time, we approximate using the last two frames.
                 prev_prev_landmarks = sequence[i-2].reshape(-1, 3)
                 prev_velocity = prev_landmarks - prev_prev_landmarks
                 acceleration = velocity - prev_velocity
                 accelerations.extend(acceleration.flatten().tolist())
            else:
                 accelerations.extend([0.0] * (num_all_points * 3)) # Zero acceleration for frame 1
        else:
            velocities.extend([0.0] * (num_all_points * 3)) # Zero velocity for frame 0
            accelerations.extend([0.0] * (num_all_points * 3)) # Zero acceleration for frame 0

        frame_features.extend(velocities)
        frame_features.extend(accelerations)

        # Pad frame features if necessary to reach the target_feature_dim based on ALL 33 points
        current_frame_dim = len(frame_features)
        expected_frame_dim = expected_dim_if_all_points_processed # This should be the target

        if current_frame_dim < expected_frame_dim:
             padding = expected_frame_dim - current_frame_dim
             frame_features.extend([0.0] * padding)
        elif current_frame_dim > expected_frame_dim:
             frame_features = frame_features[:expected_frame_dim]


        features_list.append(frame_features)


    # Convert list of frame features to numpy array (num_frames, expected_frame_dim)
    processed_sequence = np.array(features_list, dtype=np.float32)

    # Pad or truncate the sequence to max_seq_len
    if processed_sequence.shape[0] > max_seq_len:
        processed_sequence = processed_sequence[:max_seq_len]
    elif processed_sequence.shape[0] < max_seq_len:
        padding = max_seq_len - processed_sequence.shape[0]
        pad_shape = ((0, padding), (0, 0)) # Pad rows, not columns
        processed_sequence = np.pad(processed_sequence, pad_shape, mode='constant', constant_values=0)

    # Final check on dimension before returning
    if processed_sequence.shape[1] != max_feature_dim:
         print(f"Warning: Final processed sequence feature dimension mismatch. Expected {max_feature_dim}, got {processed_sequence.shape[1]}.")
         # Attempt to pad/truncate one last time just in case
         if processed_sequence.shape[1] < max_feature_dim:
              padding = max_feature_dim - processed_sequence.shape[1]
              processed_sequence = np.pad(processed_sequence, ((0, 0), (0, padding)), mode='constant', constant_values=0)
         elif processed_sequence.shape[1] > max_feature_dim:
              processed_sequence = processed_sequence[:, :max_feature_dim]

    return processed_sequence

def calculate_angle_3d(p1, v, p2):
    """Calculates the angle between three 3D points (angle at vertex v)."""
    vec1 = p1 - v
    vec2 = p2 - v

    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)

    if norm1 == 0 or norm2 == 0:
        return 0.0 # Handle zero vectors

    dot_product = np.dot(vec1, vec2)
    cosine_angle = dot_product / (norm1 * norm2)

    # Clamp cosine_angle to avoid issues with floating point inaccuracies
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    angle_radians = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees

def angle_between_vectors_3d(vec1, vec2):
    """Calculates the angle between two 3D vectors."""
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)

    if norm1 == 0 or norm2 == 0:
        return 0.0

    dot_product = np.dot(vec1, vec2)
    cosine_angle = dot_product / (norm1 * norm2)

    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    angle_radians = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees


# --- Real-time Prediction Logic ---

def predict_realtime(model, mp_pose, loaded_idx_to_exercise, loaded_idx_to_group, loaded_exercise_points, max_seq_len, max_feature_dim):
    """
    Captures video from webcam, extracts pose landmarks, preprocesses,
    and predicts exercise in real-time.
    """
    cap = cv2.VideoCapture(0) # Use webcam 0
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # Buffer to hold a sequence of recent landmark frames
    # We need max_seq_len frames for the model input
    # Store raw normalized landmarks (before adding angles/dynamics)
    landmark_buffer = deque(maxlen=max_seq_len)

    # Use the same input dimensions and mappings as loaded
    input_dim = max_feature_dim # Use the calculated max_feature_dim from loaded mappings
    num_groups = len(loaded_idx_to_group)
    num_exercises = len(loaded_idx_to_exercise)

    # Placeholder for displaying prediction
    current_prediction_exercise = "Detecting..."
    current_prediction_group = ""
    last_prediction_time = time.time()
    prediction_interval = 0.5 # Time in seconds to update prediction display

    with mp_pose.Pose(
        min_detection_confidence=MP_CONFIG['min_detection_confidence'],
        min_tracking_confidence=MP_CONFIG['min_tracking_confidence'],
        model_complexity=MP_CONFIG['model_complexity'],
        static_image_mode=MP_CONFIG['static_image_mode'],
        enable_segmentation=MP_CONFIG['enable_segmentation'],
        smooth_landmarks=MP_CONFIG['smooth_landmarks'],
        smooth_segmentation=MP_CONFIG['smooth_segmentation']
    ) as pose:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # Flip the image horizontally for a selfie-view display.
            image = cv2.flip(image, 1)

            # Convert the BGR image to RGB.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                # Extract and normalize landmarks
                frame_height, frame_width, _ = image.shape
                normalized_landmarks = extract_and_normalize_landmarks(results, frame_width, frame_height)

                if normalized_landmarks is not None:
                    # Add normalized landmarks to the buffer
                    landmark_buffer.append(normalized_landmarks.flatten()) # Store flattened 3D points

                    # If the buffer is full (or has enough frames for sequence), process
                    if len(landmark_buffer) >= max_seq_len / 2: # Process when buffer is half full or more

                        # Convert buffer to numpy array (num_frames_in_buffer, num_all_points * 3)
                        # We need to pass the buffer to the feature calculation function
                        buffer_np = np.array(landmark_buffer, dtype=np.float32)

                        # Calculate full features for the sequence in the buffer
                        # This needs to be optimized for real-time performance
                        processed_sequence_np = calculate_additional_features(
                            buffer_np,
                            loaded_exercise_points,
                            max_seq_len, # Target sequence length
                            max_feature_dim # Target feature dimension per frame
                        )

                        # Convert to Tensor, add batch dimension, and move to device
                        sequence_tensor = torch.from_numpy(processed_sequence_np).unsqueeze(0).to(device)


                        # Perform prediction periodically
                        if time.time() - last_prediction_time > prediction_interval:
                            with torch.no_grad():
                                group_outputs, exercise_outputs = model(sequence_tensor)

                                # Get predictions
                                _, predicted_group_idx = torch.max(group_outputs, 1)
                                _, predicted_exercise_idx = torch.max(exercise_outputs, 1)

                                # Convert indices to names
                                current_prediction_group = loaded_idx_to_group.get(predicted_group_idx.item(), "Unknown Group")
                                current_prediction_exercise = loaded_idx_to_exercise.get(predicted_exercise_idx.item(), "Unknown Exercise")

                                last_prediction_time = time.time()

                else:
                     current_prediction_exercise = "No Pose Detected"
                     current_prediction_group = ""


            else:
                current_prediction_exercise = "No Pose Detected"
                current_prediction_group = ""
                landmark_buffer.clear() # Clear buffer if pose is lost

            # Display the prediction on the image
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, f"Group: {current_prediction_group}", (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f"Exercise: {current_prediction_exercise}", (10, 70), font, 1, (0, 255, 0), 2, cv2.LINE_AA)


            # Display the image in Jupyter
            # In Jupyter, you typically need to use libraries like `ipywidgets` or `IPython.display`
            # for real-time video display. OpenCV's `imshow` is blocking and not ideal.
            # For a simple test, we can save frames or use a helper function.

            # Example using cv2.imshow (will likely block your notebook kernel)
            # This is more suitable for a script outside Jupyter
            cv2.imshow('MediaPipe Pose', image)


            # To make it work better in Jupyter with less blocking,
            # consider using `cv2_imshow` from `google.colab.patches` if you are in Colab,
            # or `display(Image(data=buffer))` from `IPython.display` after converting
            # the image to a suitable format (e.g., PNG or JPEG).

            # For this example, let's add a simple check for key press to exit.
            # This still requires a window manager, which might not be available
            # in headless Jupyter/Colab environments.
            key = cv2.waitKey(5) & 0xFF
            if key == ord('q') or key == 27: # 'q' or Escape key
                break

            # Alternatively, for Jupyter/Colab, you'd use a display function:
            # from google.colab.patches import cv2_imshow
            # cv2_imshow(image)


    cap.release()
    cv2.destroyAllWindows()
    print("Webcam session ended.")

# --- Execute Real-time Prediction ---

# Ensure all necessary variables are defined from the loaded mappings
if 'model' in locals() and 'loaded_idx_to_exercise' in locals() and 'mp_pose' in locals() and 'max_seq_len' in locals() and 'max_feature_dim' in locals():
    print("\nStarting real-time prediction...")
    predict_realtime(
        model=model,
        mp_pose=mp_pose, # Pass the MediaPipe pose object
        loaded_idx_to_exercise=loaded_idx_to_exercise,
        loaded_idx_to_group=loaded_idx_to_group,
        loaded_exercise_points=loaded_exercise_points, # Pass points mapping for feature calculation
        max_seq_len=max_seq_len, # Use max_seq_len from loaded config or a default
        max_feature_dim=loaded_input_dim # Use the calculated input dimension
    )
else:
    print("Error: Model, mappings, or MediaPipe Pose object not available. Please run the loading cells.")

print("\nReal-time prediction process finished.")


Error: Mappings or model file not found. Please ensure pose_classification_mappings1.json and best_pose_classification_model1.pth exist.
Error: Model file not found at best_pose_classification_model1.pth. Cannot proceed.
Error: Model, mappings, or MediaPipe Pose object not available. Please run the loading cells.

Real-time prediction process finished.


In [1]:
!pip uninstall -y tensorflow keras scipy
!pip install numpy==1.23.5 protobuf==4.25.8 mediapipe==0.10.21
!pip install tensorflow==2.15.0 keras==2.15.0 scipy==1.11.4

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
  Attempting uninstall: ml-dtypes
    Found existing installation: m